In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install datasets==2.14.7

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x8

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
!pip install transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
import transformers
transformers.__version__

'4.31.0'

In [ ]:
import inspect

from __future__ import annotations

import math
import warnings
from dataclasses import dataclass
from typing import Dict, Optional, Tuple, Union

import numpy as np
import tensorflow as tf

from transformers.activations_tf import get_tf_activation
from transformers.modeling_tf_outputs import (
    TFBaseModelOutput,
    TFMaskedLMOutput,
    TFMultipleChoiceModelOutput,
    TFQuestionAnsweringModelOutput,
    TFSequenceClassifierOutput,
    TFTokenClassifierOutput,
)
from transformers.modeling_tf_utils import (
    TFMaskedLanguageModelingLoss,
    TFModelInputType,
    TFMultipleChoiceLoss,
    TFPreTrainedModel,
    TFQuestionAnsweringLoss,
    TFSequenceClassificationLoss,
    TFTokenClassificationLoss,
    get_initializer,
    keras_serializable,
    unpack_inputs,
)
from transformers.tf_utils import check_embeddings_within_bounds, shape_list, stable_softmax
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.models.roberta.configuration_roberta import RobertaConfig

In [ ]:
class LNLFBertConfig(RobertaConfig):

    model_type = "lnlfbert"
    def __init__(
        self,
        type_sen_size=5,
        max_sen_position=64,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.type_sen_size = type_sen_size
        self.max_sen_position = max_sen_position

In [ ]:
class TFLNLFBertWordEmbeddings(tf.keras.layers.Layer):
    """
    Same as BertEmbeddings with a tiny tweak for positional embeddings indexing.
    """

    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.padding_idx = 1
        self.config = config
        self.hidden_size = config.hidden_size
        self.max_position_embeddings = config.max_position_embeddings
        self.initializer_range = config.initializer_range
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope("word_embeddings"):
            self.weight = self.add_weight(
                name="weight",
                shape=[self.config.vocab_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("token_type_embeddings"):
            self.token_type_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.config.type_vocab_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        super().build(input_shape)

    def create_position_ids_from_input_ids(self, input_ids):
        """
        Replace non-padding symbols with their position numbers. Position numbers begin at padding_idx+1. Padding
        symbols are ignored. This is modified from fairseq's `utils.make_positions`.

        Args:
            input_ids: tf.Tensor
        Returns: tf.Tensor
        """
        mask = tf.cast(tf.math.not_equal(input_ids, self.padding_idx), dtype=input_ids.dtype)
        incremental_indices = (tf.math.cumsum(mask, axis=-1)) * mask

        return incremental_indices + self.padding_idx

    def call(
        self,
        input_ids=None,
        position_ids=None,
        token_type_ids=None,
        inputs_embeds=None,
        training=False,
    ):
        """
        Applies embedding based on inputs tensor.

        Returns:
            final_embeddings (`tf.Tensor`): output embedding tensor.
        """
        assert not (input_ids is None and inputs_embeds is None)

        if input_ids is not None:
            check_embeddings_within_bounds(input_ids, self.config.vocab_size)
            inputs_embeds = tf.gather(params=self.weight, indices=input_ids)

        input_shape = shape_list(inputs_embeds)[:-1]

        if token_type_ids is None:
            token_type_ids = tf.fill(dims=input_shape, value=0)

        if position_ids is None:
            if input_ids is not None:
                # Create the position ids from the input token ids. Any padded tokens remain padded.
                position_ids = self.create_position_ids_from_input_ids(
                    input_ids=input_ids
                )
            else:
                position_ids = tf.expand_dims(
                    tf.range(start=self.padding_idx + 1, limit=input_shape[-1] + self.padding_idx + 1), axis=0
                )
                position_ids = tf.expand_dims(
                    position_ids, axis=0
                )


        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        token_type_embeds = tf.gather(params=self.token_type_embeddings, indices=token_type_ids)
        final_embeddings = inputs_embeds + position_embeds + token_type_embeds
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings, training=training)

        return final_embeddings

In [ ]:
class TFLNLFBertSenEmbeddings(tf.keras.layers.Layer):
    """
    Same as BertEmbeddings with a tiny tweak for positional embeddings indexing.
    """

    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)

        self.padding_idx = 1
        self.config = config
        self.hidden_size = config.hidden_size
        self.max_position_embeddings = config.max_sen_position
        self.initializer_range = config.initializer_range
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)
        self.concat_layer = tf.keras.layers.Concatenate(axis=1)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope('cls_embeddings'):
            self.cls_weights = self.add_weight(
                name="cls_weight",
                shape=[self.hidden_size,],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("token_type_embeddings"):
            self.token_type_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.config.type_sen_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )

        super().build(input_shape)

    def create_position_ids_from_input_ids(self, input_ids):
        """
        Replace non-padding symbols with their position numbers. Position numbers begin at padding_idx+1. Padding
        symbols are ignored. This is modified from fairseq's `utils.make_positions`.

        Args:
            input_ids: tf.Tensor
        Returns: tf.Tensor
        """
        mask = tf.cast(tf.math.not_equal(input_ids, self.padding_idx), dtype=input_ids.dtype)
        incremental_indices = (tf.math.cumsum(mask, axis=1)) * mask

        return incremental_indices + self.padding_idx

    def call(
        self,
        position_ids=None,
        token_type_ids=None,
        inputs_embeds=None,
        training=False,
    ):
        """
        Applies embedding based on inputs tensor.

        Returns:
            final_embeddings (`tf.Tensor`): output embedding tensor.
        """
        assert not (inputs_embeds is None)

        input_embed_shape = shape_list(inputs_embeds)
        input_shape = input_embed_shape[:-1]

        if token_type_ids is None:
            token_type_ids = tf.fill(dims=input_shape, value=0)

        if position_ids is None:
            position_ids = tf.expand_dims(
                tf.range(start=self.padding_idx + 1, limit=input_shape[-1] + self.padding_idx + 1), axis=0
            )

        input_embed_shape = [input_embed_shape[0], 1, input_embed_shape[2]]
        cls_tile = tf.fill(dims=input_embed_shape, value=0)
        cls_tile = tf.cast(cls_tile, self.cls_weights.dtype)
        cls_embeds = cls_tile + self.cls_weights

        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        token_type_embeds = tf.gather(params=self.token_type_embeddings, indices=token_type_ids)
        final_embeddings = inputs_embeds + position_embeds + token_type_embeds
        final_embeddings = self.concat_layer([cls_embeds, final_embeddings])
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings, training=training)

        return final_embeddings

In [ ]:
class TFLNLFBertWordPooler(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size,
            kernel_initializer=get_initializer(config.initializer_range),
            activation="tanh",
            name="dense",
        )

    def call(self, hidden_states: tf.Tensor) -> tf.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, :, 0]
        pooled_output = self.dense(inputs=first_token_tensor)

        return pooled_output

In [ ]:
class TFLNLFBertSenPooler(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size,
            kernel_initializer=get_initializer(config.initializer_range),
            activation="tanh",
            name="dense",
        )

    def call(self, hidden_states: tf.Tensor) -> tf.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(inputs=first_token_tensor)

        return pooled_output

In [ ]:
class TFLNLFBertWordSelfAttention(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number "
                f"of attention heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        self.sqrt_att_head_size = math.sqrt(self.attention_head_size)

        self.query = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="query"
        )
        self.key = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="key"
        )
        self.value = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="value"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.attention_probs_dropout_prob)

    def transpose_for_scores(self, tensor: tf.Tensor, batch_size: int, q_sen: int) -> tf.Tensor:
        # Reshape from [batch_size,  q_sen, sen_length, all_head_size] to [batch_size, q_sen, sen_length, num_attention_heads, attention_head_size]
        tensor = tf.reshape(tensor=tensor, shape=(batch_size, q_sen, -1, self.num_attention_heads, self.attention_head_size))

        # Transpose the tensor from [batch_size, q_sen, sen_length, num_attention_heads, attention_head_size] to [batch_size, q_sen, num_attention_heads, sen_length, attention_head_size]
        return tf.transpose(tensor, perm=[0, 1, 3, 2, 4])

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        batch_size, q_sen, q_length, dim = shape_list(hidden_states)
        mixed_query_layer = self.query(inputs=hidden_states)

        key_layer = self.transpose_for_scores(self.key(inputs=hidden_states), batch_size, q_sen)
        value_layer = self.transpose_for_scores(self.value(inputs=hidden_states), batch_size, q_sen)

        query_layer = self.transpose_for_scores(mixed_query_layer, batch_size, q_sen)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        # (batch size, num_heads, sen_len_q, sen_len_k)
        attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
        dk = tf.cast(self.sqrt_att_head_size, dtype=attention_scores.dtype)
        attention_scores = tf.divide(attention_scores, dk)

        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in TFLNLFBertModel call() function)
            attention_scores = tf.add(attention_scores, attention_mask)

        # Normalize the attention scores to probabilities.
        attention_probs = stable_softmax(logits=attention_scores, axis=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(inputs=attention_probs, training=training)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = tf.multiply(attention_probs, head_mask)

        attention_output = tf.matmul(attention_probs, value_layer)
        attention_output = tf.transpose(attention_output, perm=[0, 1, 3, 2, 4])

        # (batch_size, sen_len_q, all_head_size)
        attention_output = tf.reshape(tensor=attention_output, shape=(batch_size, q_sen, -1, self.all_head_size))
        outputs = (attention_output, attention_probs) if output_attentions else (attention_output,)

        return outputs

In [ ]:
class TFLNLFBertSenSelfAttention(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number "
                f"of attention heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size
        self.sqrt_att_head_size = math.sqrt(self.attention_head_size)

        self.query = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="query"
        )
        self.key = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="key"
        )
        self.value = tf.keras.layers.Dense(
            units=self.all_head_size, kernel_initializer=get_initializer(config.initializer_range), name="value"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.attention_probs_dropout_prob)

    def transpose_for_scores(self, tensor: tf.Tensor, batch_size: int) -> tf.Tensor:
        # Reshape from [batch_size, doc_length, all_head_size] to [batch_size, doc_length, num_attention_heads, attention_head_size]
        tensor = tf.reshape(tensor=tensor, shape=(batch_size, -1, self.num_attention_heads, self.attention_head_size))

        # Transpose the tensor from [batch_size, doc_length, num_attention_heads, attention_head_size] to [batch_size, num_attention_heads, doc_length, attention_head_size]
        return tf.transpose(tensor, perm=[0, 2, 1, 3])

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        batch_size = shape_list(hidden_states)[0]
        mixed_query_layer = self.query(inputs=hidden_states)

        key_layer = self.transpose_for_scores(self.key(inputs=hidden_states), batch_size)
        value_layer = self.transpose_for_scores(self.value(inputs=hidden_states), batch_size)

        query_layer = self.transpose_for_scores(mixed_query_layer, batch_size)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        # (batch size, num_heads, sen_len_q, sen_len_k)
        attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
        dk = tf.cast(self.sqrt_att_head_size, dtype=attention_scores.dtype)
        attention_scores = tf.divide(attention_scores, dk)

        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in TFRobertaModel call() function)
            attention_scores = tf.add(attention_scores, attention_mask)

        # Normalize the attention scores to probabilities.
        attention_probs = stable_softmax(logits=attention_scores, axis=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(inputs=attention_probs, training=training)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = tf.multiply(attention_probs, head_mask)

        attention_output = tf.matmul(attention_probs, value_layer)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])

        # (batch_size, sen_len_q, all_head_size)
        attention_output = tf.reshape(tensor=attention_output, shape=(batch_size, -1, self.all_head_size))
        outputs = (attention_output, attention_probs) if output_attentions else (attention_output,)

        return outputs

In [ ]:
class TFLNLFBertWordSelfOutput(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = hidden_states + input_tensor

        return hidden_states

In [ ]:
class TFLNLFBertSenSelfOutput(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = hidden_states + input_tensor

        return hidden_states

In [ ]:
class TFLNLFBertWordAttention(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.self_attention = TFLNLFBertWordSelfAttention(config, name="self")
        self.dense_output = TFLNLFBertWordSelfOutput(config, name="output")
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")

    def prune_heads(self, heads):
        raise NotImplementedError

    def call(
        self,
        input_tensor: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        hidden_states = self.LayerNorm(inputs=input_tensor)
        self_outputs = self.self_attention(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self.dense_output(
            hidden_states=self_outputs[0], input_tensor=input_tensor, training=training
        )
        # add attentions (possibly with past_key_value) if we output them
        outputs = (attention_output,) + self_outputs[1:]

        return outputs


In [ ]:
class TFLNLFBertSenAttention(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.self_attention = TFLNLFBertSenSelfAttention(config, name="self")
        self.dense_output = TFLNLFBertSenSelfOutput(config, name="output")
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")

    def prune_heads(self, heads):
        raise NotImplementedError

    def call(
        self,
        input_tensor: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        hidden_states = self.LayerNorm(inputs=input_tensor)
        self_outputs = self.self_attention(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self.dense_output(
            hidden_states=self_outputs[0], input_tensor=input_tensor, training=training
        )
        # add attentions (possibly with past_key_value) if we output them
        outputs = (attention_output,) + self_outputs[1:]

        return outputs


In [ ]:
class TFLNLFBertWordIntermediate(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.intermediate_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = get_tf_activation(config.hidden_act)
        else:
            self.intermediate_act_fn = config.hidden_act

    def call(self, hidden_states: tf.Tensor) -> tf.Tensor:
        hidden_states = self.LayerNorm(inputs=hidden_states)
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)

        return hidden_states


In [ ]:
class TFLNLFBertSenIntermediate(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.intermediate_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        if isinstance(config.hidden_act, str):
            self.intermediate_act_fn = get_tf_activation(config.hidden_act)
        else:
            self.intermediate_act_fn = config.hidden_act

    def call(self, hidden_states: tf.Tensor) -> tf.Tensor:
        hidden_states = self.LayerNorm(inputs=hidden_states)
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)

        return hidden_states


In [ ]:
class TFLNLFBertWordOutput(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = hidden_states + input_tensor

        return hidden_states

In [ ]:
class TFLNLFBertSenOutput(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.dense = tf.keras.layers.Dense(
            units=config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.dropout = tf.keras.layers.Dropout(rate=config.hidden_dropout_prob)

    def call(self, hidden_states: tf.Tensor, input_tensor: tf.Tensor, training: bool = False) -> tf.Tensor:
        hidden_states = self.dense(inputs=hidden_states)
        hidden_states = self.dropout(inputs=hidden_states, training=training)
        hidden_states = hidden_states + input_tensor

        return hidden_states

In [ ]:
class TFLNLFBertWordLayer(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.attention = TFLNLFBertWordAttention(config, name="attention")
        self.intermediate = TFLNLFBertWordIntermediate(config, name="intermediate")
        self.bert_output = TFLNLFBertWordOutput(config, name="output")

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attention_outputs = self.attention(
            input_tensor=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        intermediate_output = self.intermediate(hidden_states=attention_output)
        layer_output = self.bert_output(
            hidden_states=intermediate_output, input_tensor=attention_output, training=training
        )
        outputs = (layer_output,) + outputs  # add attentions if we output them

        return outputs


In [ ]:
class TFLNLFBertSenLayer(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)

        self.attention = TFLNLFBertSenAttention(config, name="attention")
        self.intermediate = TFLNLFBertSenIntermediate(config, name="intermediate")
        self.bert_output = TFLNLFBertSenOutput(config, name="output")

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        output_attentions: bool,
        training: bool = False,
    ) -> Tuple[tf.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attention_outputs = self.attention(
            input_tensor=hidden_states,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            training=training,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        intermediate_output = self.intermediate(hidden_states=attention_output)
        layer_output = self.bert_output(
            hidden_states=intermediate_output, input_tensor=attention_output, training=training
        )
        outputs = (layer_output,) + outputs  # add attentions if we output them

        return outputs


In [ ]:
@dataclass
class TFSenBaseModelOutput(ModelOutput):
    """
    Base class for model's outputs, with potential hidden states and attentions.

    Args:
        last_hidden_state (`tf.Tensor` of shape `(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.
        hidden_states (`tuple(tf.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `tf.Tensor` (one for the output of the embeddings + one for the output of each layer) of shape
            `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (`tuple(tf.Tensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `tf.Tensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """
    last_hidden_state: tf.Tensor = None
    last_sen_hidden_states: tf.Tensor = None
    last_first_sen_hidden_states: tf.Tensor = None
    hidden_states: Tuple[tf.Tensor] | None = None
    sen_hidden_states: Tuple[tf.Tensor] | None = None
    attentions: Tuple[tf.Tensor] | None = None
    sen_attentions: Tuple[tf.Tensor] | None = None

In [ ]:
class TFLNLFBertEncoder(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.word_layer = [TFLNLFBertWordLayer(config, name=f"layer_._{i}") for i in range(config.num_hidden_layers - 1)]
        self.sen_layer = [TFLNLFBertSenLayer(config, name=f"sen_layer_._{i}") for i in range(config.num_hidden_layers - 1)]
        self.concat_layer = tf.keras.layers.Concatenate(axis=2)
        self.sen_concat_layer = tf.keras.layers.Concatenate(axis=1)

    def call(
        self,
        hidden_states: tf.Tensor,
        attention_mask: tf.Tensor,
        head_mask: tf.Tensor,
        sen_hidden_states: tf.Tensor,
        first_sen_hidden_states: tf.Tensor,
        sen_attention_mask: tf.Tensor,
        sen_head_mask: tf.Tensor,
        output_attentions: bool,
        output_hidden_states: bool,
        return_dict: bool,
        training: bool = False,
    ) -> Union[TFSenBaseModelOutput, Tuple[tf.Tensor]]:
        all_hidden_states = () if output_hidden_states else None
        all_sen_hidden_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None
        all_sen_attentions = () if output_attentions else None

        for i, (word_layer_module, sen_layer_module) in enumerate(zip(self.word_layer, self.sen_layer)):
            if output_hidden_states:
                all_sen_hidden_states = all_sen_hidden_states + (sen_hidden_states,)

            sen_hidden_states = self.sen_concat_layer([tf.expand_dims(first_sen_hidden_states, axis=1), sen_hidden_states])

            layer_outputs = sen_layer_module(
                hidden_states=sen_hidden_states,
                attention_mask=sen_attention_mask,
                head_mask=sen_head_mask[i],
                output_attentions=output_attentions,
                training=training,
            )
            sen_hidden_states = layer_outputs[0]
            first_sen_hidden_states = sen_hidden_states[:, 0]
            sen_hidden_states = sen_hidden_states[:, 1:]

            if output_attentions:
                all_sen_attentions = all_sen_attentions + (layer_outputs[2],)


            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            hidden_states = hidden_states[:, :, 1:]
            hidden_states = self.concat_layer([tf.expand_dims(sen_hidden_states, axis=2), hidden_states])

            layer_outputs = word_layer_module(
                hidden_states=hidden_states,
                attention_mask=attention_mask,
                head_mask=head_mask[i],
                output_attentions=output_attentions,
                training=training,
            )
            hidden_states = layer_outputs[0]
            sen_hidden_states = hidden_states[:, :, 0]

            if output_attentions:
                all_attentions = all_attentions + (layer_outputs[1],)



        # Add last layer
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        if not return_dict:
            return tuple(
                v for v in [hidden_states, sen_hidden_states, all_hidden_states, all_attentions, None] if v is not None
            )

        return TFSenBaseModelOutput(
            last_hidden_state=hidden_states,
            last_sen_hidden_states=sen_hidden_states,
            last_first_sen_hidden_states=first_sen_hidden_states,
            hidden_states=all_hidden_states,
            sen_hidden_states=all_sen_hidden_states,
            attentions=all_attentions,
            sen_attentions=all_sen_attentions,
        )

In [ ]:
class TFLNLFBertEmbeddings(tf.keras.layers.Layer):
    def __init__(self, config: LNLFBertConfig, **kwargs):
        super().__init__(**kwargs)
        self.config = config
        self.word_embeddings = TFLNLFBertWordEmbeddings(config, name="word_embeddings")
        self.word_layer = TFLNLFBertWordLayer(config, name="word_layer")
        self.sen_embeddings = TFLNLFBertSenEmbeddings(config, name="sen_embeddings")

    def call(
        self,
        input_ids=None,
        position_ids=None,
        token_type_ids=None,
        inputs_embeds=None,
        attention_mask=None,
        head_mask=None,
        sen_position_ids=None,
        sen_token_type_ids=None,
        training=False,
    ):
        embedding_output = self.word_embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            training=training,
        )
        layer_outputs = self.word_layer(
            hidden_states=embedding_output,
            attention_mask=attention_mask,
            head_mask=head_mask,
            output_attentions=False,
            training=training,
        )
        hidden_states = layer_outputs[0]
        outputs = self.sen_embeddings(
            position_ids=sen_position_ids,
            token_type_ids=sen_token_type_ids,
            inputs_embeds=hidden_states[:,:,0],
            training=training,
        )
        sen_hidden_states=outputs
        first_sen_hidden_states=sen_hidden_states[:, 0]
        sen_hidden_states=sen_hidden_states[:, 1:]
        return (hidden_states, sen_hidden_states, first_sen_hidden_states)


In [ ]:
@dataclass
class TFLNLFBertBaseModelOutputWithPooling(ModelOutput):
    """
    Base class for model's outputs that also contains a pooling of the last hidden states.

    Args:
        last_hidden_state (`tf.Tensor` of shape `(batch_size, sequence_length, hidden_size)`):
            Sequence of hidden-states at the output of the last layer of the model.
        pooler_output (`tf.Tensor` of shape `(batch_size, hidden_size)`):
            Last layer hidden-state of the first token of the sequence (classification token) further processed by a
            Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence
            prediction (classification) objective during pretraining.

            This output is usually *not* a good summary of the semantic content of the input, you're often better with
            averaging or pooling the sequence of hidden-states for the whole input sequence.
        hidden_states (`tuple(tf.Tensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `tf.Tensor` (one for the output of the embeddings + one for the output of each layer) of shape
            `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (`tuple(tf.Tensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `tf.Tensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """
    last_hidden_state: tf.Tensor = None
    pooler_output: tf.Tensor = None
    last_sen_hidden_states: tf.Tensor = None
    last_first_sen_hidden_states: tf.Tensor = None
    hidden_states: Tuple[tf.Tensor] | None = None
    sen_hidden_states: Tuple[tf.Tensor] | None = None
    attentions: Tuple[tf.Tensor] | None = None
    sen_attentions: Tuple[tf.Tensor] | None = None

In [ ]:
@keras_serializable
class TFLNLFBertMainLayer(tf.keras.layers.Layer):
    config_class = LNLFBertConfig

    def __init__(self, config, add_pooling_layer=True, **kwargs):
        super().__init__(**kwargs)

        self.config = config
        self.is_decoder = config.is_decoder

        self.num_hidden_layers = config.num_hidden_layers
        self.initializer_range = config.initializer_range
        self.output_attentions = config.output_attentions
        self.output_hidden_states = config.output_hidden_states
        self.return_dict = config.use_return_dict
        self.encoder = TFLNLFBertEncoder(config, name="encoder")
        self.pooler = TFLNLFBertSenPooler(config, name="pooler") if add_pooling_layer else None
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="LayerNorm")
        # The embeddings must be the last declaration in order to follow the weights order
        self.embeddings = TFLNLFBertEmbeddings(config, name="mix_embeddings")

    # Copied from transformers.models.bert.modeling_tf_bert.TFBertMainLayer.get_input_embeddings
    def get_input_embeddings(self) -> tf.keras.layers.Layer:
        return self.embeddings

    # Copied from transformers.models.bert.modeling_tf_bert.TFBertMainLayer.set_input_embeddings
    def set_input_embeddings(self, value: tf.Variable):
        self.embeddings.weight = value
        self.embeddings.vocab_size = shape_list(value)[0]

    # Copied from transformers.models.bert.modeling_tf_bert.TFBertMainLayer._prune_heads
    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        raise NotImplementedError

    @unpack_inputs
    # Copied from transformers.models.bert.modeling_tf_bert.TFBertMainLayer.call
    def call(
        self,
        input_ids: TFModelInputType | None = None,
        attention_mask: np.ndarray | tf.Tensor | None = None,
        sen_attention_mask: np.ndarray | tf.Tensor | None = None,
        token_type_ids: np.ndarray | tf.Tensor | None = None,
        sen_token_type_ids: np.ndarray | tf.Tensor | None = None,
        position_ids: np.ndarray | tf.Tensor | None = None,
        sen_position_ids: np.ndarray | tf.Tensor | None = None,
        head_mask: np.ndarray | tf.Tensor | None = None,
        sen_head_mask: np.ndarray | tf.Tensor | None = None,
        inputs_embeds: np.ndarray | tf.Tensor | None = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        training: bool = False,
    ) -> Union[TFLNLFBertBaseModelOutputWithPooling, Tuple[tf.Tensor]]:
        if not self.config.is_decoder:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = shape_list(input_ids)
        elif inputs_embeds is not None:
            input_shape = shape_list(inputs_embeds)[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, doc_length, sen_length = input_shape

        if attention_mask is None:
            attention_mask = tf.fill(dims=(batch_size, doc_length, sen_length), value=1)

        if sen_attention_mask is None:
            sen_attention_mask = tf.fill(dims=(batch_size, doc_length), value=1)

        if token_type_ids is None:
            token_type_ids = tf.fill(dims=input_shape, value=0)

        if sen_token_type_ids is None:
            sen_token_type_ids = tf.fill(dims=input_shape[:-1], value=0)

        attention_mask_shape = shape_list(attention_mask)

        mask_sen_length = sen_length

        extended_attention_mask = tf.reshape(
            attention_mask, (attention_mask_shape[0], attention_mask_shape[1], 1, 1, attention_mask_shape[2])
        )

        sen_attention_mask = tf.cast(sen_attention_mask, dtype=tf.float32)
        cls_mask = tf.ones((shape_list(sen_attention_mask)[0], 1))

        cls_mask = tf.cast(cls_mask, dtype=tf.float32)

        sen_attention_mask = tf.concat([cls_mask, sen_attention_mask], 1)

        sen_attention_mask_shape = shape_list(sen_attention_mask)

        extended_sen_attention_mask = tf.reshape(
            sen_attention_mask, (sen_attention_mask_shape[0], 1, 1, sen_attention_mask_shape[1])
        )

        extended_attention_mask = tf.cast(extended_attention_mask, dtype=tf.float32)
        extended_sen_attention_mask = tf.cast(extended_sen_attention_mask, dtype=tf.float32)
        one_cst = tf.constant(1.0, dtype=tf.float32)
        ten_thousand_cst = tf.constant(-10000.0, dtype=tf.float32)
        extended_attention_mask = tf.multiply(tf.subtract(one_cst, extended_attention_mask), ten_thousand_cst)
        extended_sen_attention_mask = tf.multiply(tf.subtract(one_cst, extended_sen_attention_mask), ten_thousand_cst)

        encoder_extended_attention_mask = None

        if head_mask is not None:
            raise NotImplementedError
        else:
            head_mask = [None] * self.config.num_hidden_layers
            sen_head_mask = [None] * (self.config.num_hidden_layers - 1)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            sen_position_ids=sen_position_ids,
            token_type_ids=token_type_ids,
            sen_token_type_ids=sen_token_type_ids,
            inputs_embeds=inputs_embeds,
            attention_mask=extended_attention_mask,
            head_mask=head_mask[0],
            training=training,
        )

        encoder_outputs = self.encoder(
            hidden_states=embedding_output[0],
            attention_mask=extended_attention_mask,
            head_mask=head_mask[1:],
            sen_hidden_states=embedding_output[1],
            first_sen_hidden_states=embedding_output[2],
            sen_attention_mask=extended_sen_attention_mask,
            sen_head_mask=sen_head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            training=training,
        )

        sequence_output = encoder_outputs[0]
        sequence_output = self.LayerNorm(sequence_output)
        pooled_output = self.pooler(hidden_states=sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (
                sequence_output,
                pooled_output,
            ) + encoder_outputs[1:]



        return TFLNLFBertBaseModelOutputWithPooling(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            last_sen_hidden_states=encoder_outputs.last_sen_hidden_states,
            last_first_sen_hidden_states=encoder_outputs.last_first_sen_hidden_states,
            hidden_states=encoder_outputs.hidden_states,
            sen_hidden_states=encoder_outputs.sen_hidden_states,
            attentions=encoder_outputs.attentions,
            sen_attentions=encoder_outputs.sen_attentions,
        )

In [ ]:
class TFLNLFBertPreTrainedModel(TFPreTrainedModel):
    """
    An abstract class to handle weights initialization and a simple interface for downloading and loading pretrained
    models.
    """

    config_class = LNLFBertConfig
    base_model_prefix = "lnlfbert"

    @property
    def input_signature(self) -> Dict[str, tf.TensorSpec]:
      model_inputs = list(inspect.signature(self.call).parameters)
      sig = {}
      for input_name in (
              "input_ids",
              "attention_mask",
              "decoder_input_ids",
              "decoder_attention_mask",
          ):
              if input_name in model_inputs:
                  sig[input_name] = tf.TensorSpec([None] * 3, tf.int32, name=input_name)

      for input_name in (
              "sen_token_type_ids",
              "sen_attention_mask",
          ):
              if input_name in model_inputs:
                  sig[input_name] = tf.TensorSpec([None] * 2, tf.int32, name=input_name)

      return sig

In [ ]:
class TFLNLFBertClassificationHead(tf.keras.layers.Layer):
    """Head for sentence-level classification tasks."""

    def __init__(self, config, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            config.hidden_size,
            kernel_initializer=get_initializer(config.initializer_range),
            activation="relu",
            name="dense",
        )
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = tf.keras.layers.Dropout(classifier_dropout)
        self.out_proj = tf.keras.layers.Dense(
            config.num_labels, kernel_initializer=get_initializer(config.initializer_range), name="out_proj"
        )

    def call(self, features, training=False):
        x = self.dropout(features, training=training)
        x = self.dense(x)
        x = self.dropout(x, training=training)
        x = self.out_proj(x)
        return x

In [ ]:
class TFLNLFBertLMHead(tf.keras.layers.Layer):
    """LNLFBert Head for masked language modeling."""

    def __init__(self, config, input_embeddings, **kwargs):
        super().__init__(**kwargs)

        self.config = config
        self.hidden_size = config.hidden_size
        self.dense = tf.keras.layers.Dense(
            config.hidden_size, kernel_initializer=get_initializer(config.initializer_range), name="dense"
        )
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=config.layer_norm_eps, name="layer_norm")
        self.act = get_tf_activation("gelu")

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        self.decoder = input_embeddings

    def build(self, input_shape):
        self.bias = self.add_weight(shape=(self.config.vocab_size,), initializer="zeros", trainable=True, name="bias")

        super().build(input_shape)

    def get_output_embeddings(self):
        return self.decoder

    def set_output_embeddings(self, value):
        self.decoder.weight = value
        self.decoder.vocab_size = shape_list(value)[0]

    def get_bias(self):
        return {"bias": self.bias}

    def set_bias(self, value):
        self.bias = value["bias"]
        self.config.vocab_size = shape_list(value["bias"])[0]

    def call(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.layer_norm(hidden_states)

        # project back to size of vocabulary with bias
        input_shape = shape_list(tensor=hidden_states)
        doc_length = input_shape[1]
        sen_length = input_shape[2]
        hidden_states = tf.reshape(tensor=hidden_states, shape=[-1, self.hidden_size])
        hidden_states = tf.matmul(a=hidden_states, b=self.decoder.weight, transpose_b=True)
        hidden_states = tf.reshape(tensor=hidden_states, shape=[-1, doc_length, sen_length, self.config.vocab_size])
        hidden_states = tf.nn.bias_add(value=hidden_states, bias=self.bias)

        return hidden_states

In [ ]:
LNLFBERT_START_DOCSTRING = r"""

    This model inherits from [`TFPreTrainedModel`]. Check the superclass documentation for the generic methods the
    library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
    etc.)

    This model is also a [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model) subclass. Use it
    as a regular TF 2.0 Keras Model and refer to the TF 2.0 documentation for all matter related to general usage and
    behavior.

    <Tip>

    TensorFlow models and layers in `transformers` accept two formats as input:

    - having all inputs as keyword arguments (like PyTorch models), or
    - having all inputs as a list, tuple or dict in the first positional argument.

    The reason the second format is supported is that Keras methods prefer this format when passing inputs to models
    and layers. Because of this support, when using methods like `model.fit()` things should "just work" for you - just
    pass your inputs and labels in any format that `model.fit()` supports! If, however, you want to use the second
    format outside of Keras methods like `fit()` and `predict()`, such as when creating your own layers or models with
    the Keras `Functional` API, there are three possibilities you can use to gather all the input Tensors in the first
    positional argument:

    - a single Tensor with `input_ids` only and nothing else: `model(input_ids)`
    - a list of varying length with one or several input Tensors IN THE ORDER given in the docstring:
    `model([input_ids, attention_mask])` or `model([input_ids, attention_mask, token_type_ids])`
    - a dictionary with one or several input Tensors associated to the input names given in the docstring:
    `model({"input_ids": input_ids, "token_type_ids": token_type_ids})`

    Note that when creating models and layers with
    [subclassing](https://keras.io/guides/making_new_layers_and_models_via_subclassing/) then you don't need to worry
    about any of this, as you can just pass inputs like you would to any other Python function!

    </Tip>

    Parameters:
        config ([`LNLFBertConfig`]): Model configuration class with all the parameters of the
            model. Initializing with a config file does not load the weights associated with the model, only the
            configuration. Check out the [`~PreTrainedModel.from_pretrained`] method to load the model weights.
"""

LNLFBERT_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`Numpy array` or `tf.Tensor` of shape `({0})`):
            Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.__call__`] and
            [`PreTrainedTokenizer.encode`] for details.

            [What are input IDs?](../glossary#input-ids)
        attention_mask (`Numpy array` or `tf.Tensor` of shape `({0})`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            [What are attention masks?](../glossary#attention-mask)
        token_type_ids (`Numpy array` or `tf.Tensor` of shape `({0})`, *optional*):
            Segment token indices to indicate first and second portions of the inputs. Indices are selected in `[0,
            1]`:

            - 0 corresponds to a *sentence A* token,
            - 1 corresponds to a *sentence B* token.

            [What are token type IDs?](../glossary#token-type-ids)
        position_ids (`Numpy array` or `tf.Tensor` of shape `({0})`, *optional*):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range `[0,
            config.max_position_embeddings - 1]`.

            [What are position IDs?](../glossary#position-ids)
        head_mask (`Numpy array` or `tf.Tensor` of shape `(num_heads,)` or `(num_layers, num_heads)`, *optional*):
            Mask to nullify selected heads of the self-attention modules. Mask values selected in `[0, 1]`:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        inputs_embeds (`tf.Tensor` of shape `({0}, hidden_size)`, *optional*):
            Optionally, instead of passing `input_ids` you can choose to directly pass an embedded representation. This
            is useful if you want more control over how to convert `input_ids` indices into associated vectors than the
            model's internal embedding lookup matrix.
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail. This argument can be used only in eager mode, in graph mode the value in the
            config will be used instead.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail. This argument can be used only in eager mode, in graph mode the value in the config will be
            used instead.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple. This argument can be used in
            eager mode, in graph mode the value will always be set to True.
        training (`bool`, *optional*, defaults to `False`):
            Whether or not to use the model in training mode (some modules like dropout modules have different
            behaviors between training and evaluation).
"""

In [ ]:
_CHECKPOINT_FOR_DOC = "lnlfbert-base"
_CONFIG_FOR_DOC = "LNLFBertConfig"

In [ ]:
@add_start_docstrings("""LNLFBert Model with a `language modeling` head on top.""", LNLFBERT_START_DOCSTRING)
class TFLNLFBertForMaskedLM(TFLNLFBertPreTrainedModel, TFMaskedLanguageModelingLoss):
    # names with a '.' represents the authorized unexpected/missing layers when a TF model is loaded from a PT model
    _keys_to_ignore_on_load_unexpected = [r"pooler", r"lm_head.decoder.weight"]

    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)

        self.lnlfbert = TFLNLFBertMainLayer(config, add_pooling_layer=False, name="lnlfbert")
        self.lm_head = TFLNLFBertLMHead(config, self.lnlfbert.embeddings.word_embeddings, name="lm_head")

    def get_lm_head(self):
        return self.lm_head

    def get_prefix_bias_name(self):
        warnings.warn("The method get_prefix_bias_name is deprecated. Please use `get_bias` instead.", FutureWarning)
        return self.name + "/" + self.lm_head.name

    def hf_compute_loss(self, labels, logits):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE
        )
        if self.config.tf_legacy_loss:
            # make sure only labels that are not equal to -100 affect the loss
            active_loss = tf.not_equal(tf.reshape(labels, (-1,)), -100)
            reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[-1])), active_loss)
            labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
            return loss_fn(labels, reduced_logits)

        # Clip negative labels to zero here to avoid NaNs and errors - those positions will get masked later anyway
        inputs_shape = shape_list(logits)
        new_labels_shape = (inputs_shape[0], -1)
        new_logits_shape = (inputs_shape[0], -1, inputs_shape[-1])
        labels = tf.reshape(labels, new_labels_shape)
        logits = tf.reshape(logits, new_logits_shape)
        unmasked_loss = loss_fn(tf.nn.relu(labels), logits)
        # make sure only labels that are not equal to -100 affect the loss
        loss_mask = tf.cast(labels != -100, dtype=unmasked_loss.dtype)
        masked_loss = unmasked_loss * loss_mask
        sum_loss_mask = tf.reduce_sum(loss_mask)
        reduced_masked_loss = tf.where(
                                tf.reshape(tf.math.greater_equal(sum_loss_mask, 0.001), (1,)),
                                tf.reshape(tf.reduce_sum(masked_loss)/sum_loss_mask, (1,)),
                                tf.cast(tf.fill([1], 0.), dtype=unmasked_loss.dtype)
                              )
        return tf.reshape(reduced_masked_loss, (1,))

    @unpack_inputs
    @add_start_docstrings_to_model_forward(LNLFBERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=TFMaskedLMOutput,
        config_class=_CONFIG_FOR_DOC,
        mask="<mask>",
        expected_output="' Paris'",
        expected_loss=0.1,
    )
    def call(
        self,
        input_ids: TFModelInputType | None = None,
        attention_mask: np.ndarray | tf.Tensor | None = None,
        sen_attention_mask: np.ndarray | tf.Tensor | None = None,
        token_type_ids: np.ndarray | tf.Tensor | None = None,
        sen_token_type_ids: np.ndarray | tf.Tensor | None = None,
        position_ids: np.ndarray | tf.Tensor | None = None,
        sen_position_ids: np.ndarray | tf.Tensor | None = None,
        head_mask: np.ndarray | tf.Tensor | None = None,
        sen_head_mask: np.ndarray | tf.Tensor | None = None,
        inputs_embeds: np.ndarray | tf.Tensor | None = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        labels: np.ndarray | tf.Tensor | None = None,
        training: Optional[bool] = False,
    ) -> Union[TFMaskedLMOutput, Tuple[tf.Tensor]]:
        r"""
        labels (`tf.Tensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should be in `[-100, 0, ...,
            config.vocab_size]` (see `input_ids` docstring) Tokens with indices set to `-100` are ignored (masked), the
            loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`
        """
        outputs = self.lnlfbert(
            input_ids,
            attention_mask=attention_mask,
            sen_attention_mask=sen_attention_mask,
            token_type_ids=token_type_ids,
            sen_token_type_ids=sen_token_type_ids,
            position_ids=position_ids,
            sen_position_ids=sen_position_ids,
            head_mask=head_mask,
            sen_head_mask=sen_head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            training=training,
        )

        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        loss = None if labels is None else self.hf_compute_loss(labels, prediction_scores)

        if not return_dict:
            output = (prediction_scores,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TFMaskedLMOutput(
            loss=loss,
            logits=prediction_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
@add_start_docstrings(
    """
    LNLFBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the
    pooled output) e.g. for GLUE tasks.
    """,
    LNLFBERT_START_DOCSTRING,
)
class TFLNLFBertForSequenceClassification(TFLNLFBertPreTrainedModel, TFSequenceClassificationLoss):
    # names with a '.' represents the authorized unexpected/missing layers when a TF model is loaded from a PT model
    _keys_to_ignore_on_load_unexpected = [r"pooler", r"lm_head"]

    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.num_labels = config.num_labels

        self.lnlfbert = TFLNLFBertMainLayer(config, add_pooling_layer=False, name="lnlfbert")
        self.classifier = TFLNLFBertClassificationHead(config, name="classifier")

    @unpack_inputs
    @add_start_docstrings_to_model_forward(LNLFBERT_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    @add_code_sample_docstrings(
        checkpoint="cardiffnlp/twitter-roberta-base-emotion",
        output_type=TFSequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output="'optimism'",
        expected_loss=0.08,
    )
    def call(
        self,
        input_ids: TFModelInputType | None = None,
        attention_mask: np.ndarray | tf.Tensor | None = None,
        sen_attention_mask: np.ndarray | tf.Tensor | None = None,
        token_type_ids: np.ndarray | tf.Tensor | None = None,
        sen_token_type_ids: np.ndarray | tf.Tensor | None = None,
        position_ids: np.ndarray | tf.Tensor | None = None,
        sen_position_ids: np.ndarray | tf.Tensor | None = None,
        head_mask: np.ndarray | tf.Tensor | None = None,
        sen_head_mask: np.ndarray | tf.Tensor | None = None,
        inputs_embeds: np.ndarray | tf.Tensor | None = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        labels: np.ndarray | tf.Tensor | None = None,
        training: Optional[bool] = False,
    ) -> Union[TFSequenceClassifierOutput, Tuple[tf.Tensor]]:
        r"""
        labels (`tf.Tensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        outputs = self.lnlfbert(
            input_ids,
            attention_mask=attention_mask,
            sen_attention_mask=sen_attention_mask,
            token_type_ids=token_type_ids,
            sen_token_type_ids=sen_token_type_ids,
            position_ids=position_ids,
            sen_position_ids=sen_position_ids,
            head_mask=head_mask,
            sen_head_mask=sen_head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            training=training,
        )
        #print("outputs: ", outputs)
        sequence_output = outputs.last_sen_hidden_states
        logits = self.classifier(sequence_output, training=training)

        loss = None if labels is None else self.hf_compute_loss(labels, logits)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return TFSequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
config = LNLFBertConfig.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
from transformers import RobertaTokenizer, RobertaTokenizerFast
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
!pip install nltk

In [ ]:
import nltk
import tensorflow as tf
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from collections.abc import Mapping, Sized
from typing import TYPE_CHECKING, Any, Dict, List, NamedTuple, Optional, Sequence, Tuple, Union
from transformers.tokenization_utils_base import BatchEncoding
from transformers.utils import (
    ExplicitEnum,
    PaddingStrategy,
    PushToHubMixin,
    TensorType,
    add_end_docstrings,
    add_model_info_to_auto_map,
    cached_file,
    copy_func,
    download_url,
    extract_commit_hash,
    is_flax_available,
    is_jax_tensor,
    is_numpy_array,
    is_offline_mode,
    is_remote_url,
    is_tf_available,
    is_tf_tensor,
    is_tokenizers_available,
    is_torch_available,
    is_torch_device,
    is_torch_tensor,
    logging,
    requires_backends,
    to_py_obj,
)
EncodedInput = List[int]

In [ ]:
class LNLFBertTokenizer(RobertaTokenizerFast):

  model_input_names: List[str] = ["input_ids", "attention_mask", "sen_token_type_ids", "sen_attention_mask", "special_tokens_mask"]

  def __call__(self, texts, padding=False, return_tensors=None, multi_texts=False, order=None, truncation=False, max_length=None, max_sen_num=None, return_special_tokens_mask=None):
    if padding:
      return self.tokenizer_with_padding(
          texts, return_tensors=return_tensors, multi_texts=multi_texts, order=order,
          truncation=truncation, max_length=max_length, max_sen_num=max_sen_num, return_special_tokens_mask=return_special_tokens_mask)
    else:
      return self.tokenizer_without_padding(
          texts, return_tensors=return_tensors, multi_texts=multi_texts, order=order,
          truncation=truncation, max_length=max_length, max_sen_num=max_sen_num, return_special_tokens_mask=return_special_tokens_mask)

  def tokenizer_without_padding(self, texts, return_tensors=None, multi_texts=False, order=None, truncation=False, max_length=None,
                                max_sen_num=None, return_special_tokens_mask=None):
    results = []
    sents_len = []
    sen_token_type_ids = []
    sen_attention_mask = []

    if multi_texts:
      list_keys = order
      for i in range(len(texts[list_keys[0]])):
        tmp_sentences = []
        #sen_attention_mask.append(1)
        for key_name in list_keys:
          text = texts[key_name][i]
          if text is None:
            text = ''
          try:
            sentences = nltk.sent_tokenize(text)
          except:
            print("An exception occurred: ",text)
          if truncation and max_sen_num:
            sen_len = min(len(sentences), max_sen_num[key_name])
            sentences = sentences[0:sen_len]
          tmp_sentences.extend(sentences)
          for sentence in sentences:
            results.append(sentence)
            sen_token_type_ids.append(key_name)
            if sentence:
              sen_attention_mask.append(1)
            else:
              sen_attention_mask.append(0)
        sents_len.append(len(tmp_sentences))
      output = super().__call__(results, return_tensors=return_tensors, truncation=truncation,
                                   max_length=max_length,return_special_tokens_mask=return_special_tokens_mask)
      for key, value in output.items():
        output[key] = self.set_sentence_len(value, sents_len)
      output['sen_attention_mask'] = self.set_sentence_len(sen_attention_mask, sents_len)
      output['sen_token_type_ids'] = self.set_sentence_len(sen_token_type_ids, sents_len)
      return output
    else:
      for text in texts:
        sentences = nltk.sent_tokenize(text)
        if truncation and max_sen_num:
          sen_len = min(len(sentences), max_sen_num)
          sentences = sentences[0:sen_len]
        sents_len.append(len(sentences))
        for sentence in sentences:
          results.append(sentence)
          if sentence:
            sen_attention_mask.append(1)
          else:
            sen_attention_mask.append(0)

      output = super().__call__(results, return_tensors=return_tensors, truncation=truncation,
                                   max_length=max_length, return_special_tokens_mask=return_special_tokens_mask)
      for key, value in output.items():
        output[key] = self.set_sentence_len(value, sents_len)
      output['sen_attention_mask'] = self.set_sentence_len(sen_attention_mask, sents_len)
      return output

  def set_mask_sentence_len(self, inputs, sents_len):
    output = []
    start_index = 0
    end_index = 0
    inputs = inputs.copy()
    for sent_len in sents_len:
      end_index = start_index + sent_len + 1
      output.append(inputs[start_index:end_index])
      start_index = end_index
    return output

  def set_sentence_len(self, inputs, sents_len):
    output = []
    start_index = 0
    end_index = 0
    inputs = inputs.copy()
    for sent_len in sents_len:
      end_index = start_index + sent_len
      output.append(inputs[start_index:end_index])
      start_index = end_index
    return output

  def tokenizer_with_padding(self, texts, return_tensors=None, multi_texts=False, order=None,
                             truncation=False, max_length=None, max_sen_num=None, return_special_tokens_mask=None):
    max_len = 0
    results = []
    temp_results = []
    temp_sen_token_type_ids = []
    sen_token_type_ids = []
    sen_attention_mask = []

    if multi_texts:
      list_keys = order
      for i in range(len(texts[list_keys[0]])):
        tmp_sentences = []
        tmp_sen_tokens = []
        for key_name in list_keys:
          text = texts[key_name][i]
          if text is None:
            text = ''
          try:
            sentences = nltk.sent_tokenize(text)
          except:
            print("An exception occurred: ",text)
          if truncation and max_sen_num:
            sen_len = min(len(sentences), max_sen_num[key_name])
            sentences = sentences[0:sen_len]
          tmp_sentences.extend(sentences)
          tmp_sen_tokens.extend([key_name] * len(sentences))
        max_len = max(max_len, len(tmp_sentences))
        temp_results.append(tmp_sentences)
        temp_sen_token_type_ids.append(tmp_sen_tokens)
      for sentences, sen_tokens in zip(temp_results, temp_sen_token_type_ids):
        #if sen_token_type_id == list_keys[0]:
          #sen_attention_mask.append(1)
        while len(sentences) < max_len:
          sentences.append("")
        for sentence, sen_token_type_id in zip(sentences, sen_tokens):
          results.append(sentence)
          sen_token_type_ids.append(sen_token_type_id)
          if sentence:
            sen_attention_mask.append(1)
          else:
            sen_attention_mask.append(0)

      output = super().__call__(results, padding=True, return_tensors=return_tensors, truncation=truncation,
                                   max_length=max_length, return_special_tokens_mask=return_special_tokens_mask)
      for key, value in output.items():
        output[key] = tf.reshape(value, [len(texts[list_keys[0]]), max_len, -1])
      output['sen_attention_mask'] = tf.reshape(sen_attention_mask, [len(texts[list_keys[0]]), -1])
      output['sen_token_type_ids'] = tf.reshape(sen_token_type_ids, [len(texts[list_keys[0]]), -1])
      return output
    else:
      for text in texts:
        sentences = nltk.sent_tokenize(text)
        if truncation and max_sen_num:
            sen_len = max(len(sentences), max_sen_num)
        else:
            sen_len = len(sentences)
        max_len = max(max_len, sen_len)
        temp_results.append(sentences[0:sen_len])
      for sentences in temp_results:
        while len(sentences) < max_len:
          sentences.append("")
        for sentence in sentences:
          results.append(sentence)
          if sentence:
            sen_attention_mask.append(1)
          else:
            sen_attention_mask.append(0)

      output = super().__call__(results, padding=True, return_tensors=return_tensors, truncation=truncation,
                                   max_length=max_length, return_special_tokens_mask=return_special_tokens_mask)
      for key, value in output.items():
        output[key] = tf.reshape(value, [len(texts), max_len, -1])
      output['sen_attention_mask'] = tf.reshape(sen_attention_mask, [len(texts), -1])
      return output

  def pad(
        self,
        encoded_inputs: Union[
            BatchEncoding,
            List[BatchEncoding],
            Dict[str, EncodedInput],
            Dict[str, List[EncodedInput]],
            List[Dict[str, EncodedInput]],
        ],
        padding: Union[bool, str, PaddingStrategy] = True,
        max_length: Optional[int] = None,
        pad_to_multiple_of: Optional[int] = None,
        return_attention_mask: Optional[bool] = None,
        return_tensors: Optional[Union[str, TensorType]] = None,
        verbose: bool = True,
    ) -> BatchEncoding:
        """
        Pad a single encoded input or a batch of encoded inputs up to predefined length or to the max sequence length
        in the batch.

        Padding side (left/right) padding token ids are defined at the tokenizer level (with `self.padding_side`,
        `self.pad_token_id` and `self.pad_token_type_id`).

        Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the
        text followed by a call to the `pad` method to get a padded encoding.

        <Tip>

        If the `encoded_inputs` passed are dictionary of numpy arrays, PyTorch tensors or TensorFlow tensors, the
        result will use the same type unless you provide a different tensor type with `return_tensors`. In the case of
        PyTorch tensors, you will lose the specific device of your tensors however.

        </Tip>

        Args:
            encoded_inputs ([`BatchEncoding`], list of [`BatchEncoding`], `Dict[str, List[int]]`, `Dict[str, List[List[int]]` or `List[Dict[str, List[int]]]`):
                Tokenized inputs. Can represent one input ([`BatchEncoding`] or `Dict[str, List[int]]`) or a batch of
                tokenized inputs (list of [`BatchEncoding`], *Dict[str, List[List[int]]]* or *List[Dict[str,
                List[int]]]*) so you can use this method during preprocessing as well as in a PyTorch Dataloader
                collate function.

                Instead of `List[int]` you can have tensors (numpy arrays, PyTorch tensors or TensorFlow tensors), see
                the note above for the return type.
            padding (`bool`, `str` or [`~utils.PaddingStrategy`], *optional*, defaults to `True`):
                 Select a strategy to pad the returned sequences (according to the model's padding side and padding
                 index) among:

                - `True` or `'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
                  sequence if provided).
                - `'max_length'`: Pad to a maximum length specified with the argument `max_length` or to the maximum
                  acceptable input length for the model if that argument is not provided.
                - `False` or `'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of different
                  lengths).
            max_length (`int`, *optional*):
                Maximum length of the returned list and optionally padding length (see above).
            pad_to_multiple_of (`int`, *optional*):
                If set will pad the sequence to a multiple of the provided value.

                This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability
                `>= 7.5` (Volta).
            return_attention_mask (`bool`, *optional*):
                Whether to return the attention mask. If left to the default, will return the attention mask according
                to the specific tokenizer's default, defined by the `return_outputs` attribute.

                [What are attention masks?](../glossary#attention-mask)
            return_tensors (`str` or [`~utils.TensorType`], *optional*):
                If set, will return tensors instead of list of python integers. Acceptable values are:

                - `'tf'`: Return TensorFlow `tf.constant` objects.
                - `'pt'`: Return PyTorch `torch.Tensor` objects.
                - `'np'`: Return Numpy `np.ndarray` objects.
            verbose (`bool`, *optional*, defaults to `True`):
                Whether or not to print more information and warnings.
        """
        if self.__class__.__name__.endswith("Fast"):
            if not self.deprecation_warnings.get("Asking-to-pad-a-fast-tokenizer", False):
                logger.warning_advice(
                    f"You're using a {self.__class__.__name__} tokenizer. Please note that with a fast tokenizer,"
                    " using the `__call__` method is faster than using a method to encode the text followed by a call"
                    " to the `pad` method to get a padded encoding."
                )
                self.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

        # If we have a list of dicts, let's convert it in a dict of lists
        # We do this to allow using this method as a collate_fn function in PyTorch Dataloader
        if isinstance(encoded_inputs, (list, tuple)) and isinstance(encoded_inputs[0], Mapping):
            encoded_inputs = {key: [example[key] for example in encoded_inputs] for key in encoded_inputs[0].keys()}

        # The model's main input name, usually `input_ids`, has be passed for padding
        if self.model_input_names[0] not in encoded_inputs:
            raise ValueError(
                "You should supply an encoding or a list of encodings to this method "
                f"that includes {self.model_input_names[0]}, but you provided {list(encoded_inputs.keys())}"
            )

        required_input = encoded_inputs[self.model_input_names[0]]

        if required_input is None or (isinstance(required_input, Sized) and len(required_input) == 0):
            if return_attention_mask:
                encoded_inputs["attention_mask"] = []
            return encoded_inputs

        # If we have PyTorch/TF/NumPy tensors/arrays as inputs, we cast them as python objects
        # and rebuild them afterwards if no return_tensors is specified
        # Note that we lose the specific device the tensor may be on for PyTorch

        first_element = required_input[0]
        if isinstance(first_element, (list, tuple)):
            # first_element might be an empty list/tuple in some edge cases so we grab the first non empty element.
            for item in required_input:
                if len(item) != 0:
                    first_element = item[0]
                    break
        # At this state, if `first_element` is still a list/tuple, it's an empty one so there is nothing to do.
        if not isinstance(first_element, (int, list, tuple)):
            if is_tf_tensor(first_element):
                return_tensors = "tf" if return_tensors is None else return_tensors
            elif is_torch_tensor(first_element):
                return_tensors = "pt" if return_tensors is None else return_tensors
            elif isinstance(first_element, np.ndarray):
                return_tensors = "np" if return_tensors is None else return_tensors
            else:
                raise ValueError(
                    f"type of {first_element} unknown: {type(first_element)}. "
                    "Should be one of a python, numpy, pytorch or tensorflow object."
                )

            for key, value in encoded_inputs.items():
                encoded_inputs[key] = to_py_obj(value)

        # Convert padding_strategy in PaddingStrategy
        padding_strategy, _, max_length, _ = super()._get_padding_truncation_strategies(
            padding=padding, max_length=max_length, verbose=verbose
        )

        required_input = encoded_inputs[self.model_input_names[0]]
        if required_input and not isinstance(required_input[0], (list, tuple)):
            encoded_inputs = self._pad(
                encoded_inputs,
                max_length=max_length,
                padding_strategy=padding_strategy,
                pad_to_multiple_of=pad_to_multiple_of,
                return_attention_mask=return_attention_mask,
            )
            return BatchEncoding(encoded_inputs, tensor_type=return_tensors)

        batch_size = len(required_input)
        assert all(
            len(v) == batch_size for v in encoded_inputs.values()
        ), "Some items in the output dictionary have a different batch size than others."

        if padding_strategy == PaddingStrategy.LONGEST:
            max_sen_length = max(len(inputs) for inputs in required_input)
            max_length = 0
            for input in required_input:
              max_length = max(max_length, max([len(x) for x in input], default=0))
            padding_strategy = PaddingStrategy.MAX_LENGTH

        batch_outputs = {}
        for i in range(batch_size):
            inputs = {k: v[i] for k, v in encoded_inputs.items()}
            outputs = self._pad(
                inputs,
                max_length=max_length,
                max_sen_length=max_sen_length,
                padding_strategy=padding_strategy,
                pad_to_multiple_of=pad_to_multiple_of,
                return_attention_mask=return_attention_mask,
            )

            for key, value in outputs.items():
                if key not in batch_outputs:
                    batch_outputs[key] = []
                batch_outputs[key].append(value)

        return BatchEncoding(batch_outputs, tensor_type=return_tensors)

  def _pad(
      self,
      encoded_inputs: Union[Dict[str, EncodedInput], BatchEncoding],
      max_length: Optional[int] = None,
      max_sen_length: Optional[int] = None,
      padding_strategy: PaddingStrategy = PaddingStrategy.DO_NOT_PAD,
      pad_to_multiple_of: Optional[int] = None,
      return_attention_mask: Optional[bool] = None,
  ) -> dict:
      """
      Pad encoded inputs (on left/right and up to predefined length or max length in the batch)

      Args:
          encoded_inputs:
              Dictionary of tokenized inputs (`List[int]`) or batch of tokenized inputs (`List[List[int]]`).
          max_length: maximum length of the returned list and optionally padding length (see below).
              Will truncate by taking into account the special tokens.
          padding_strategy: PaddingStrategy to use for padding.

              - PaddingStrategy.LONGEST Pad to the longest sequence in the batch
              - PaddingStrategy.MAX_LENGTH: Pad to the max length (default)
              - PaddingStrategy.DO_NOT_PAD: Do not pad
              The tokenizer padding sides are defined in self.padding_side:

                  - 'left': pads on the left of the sequences
                  - 'right': pads on the right of the sequences
          pad_to_multiple_of: (optional) Integer if set will pad the sequence to a multiple of the provided value.
              This is especially useful to enable the use of Tensor Core on NVIDIA hardware with compute capability
              `>= 7.5` (Volta).
          return_attention_mask:
              (optional) Set to False to avoid returning attention mask (default: set to model specifics)
      """
      # Load from model defaults
      #print(encoded_inputs)
      if return_attention_mask is None:
          return_attention_mask = "attention_mask" in self.model_input_names

      required_input = encoded_inputs[self.model_input_names[0]]

      if padding_strategy == PaddingStrategy.LONGEST:
          max_length = len(required_input)

      if max_length is not None and pad_to_multiple_of is not None and (max_length % pad_to_multiple_of != 0):
          max_length = ((max_length // pad_to_multiple_of) + 1) * pad_to_multiple_of

      needs_to_be_padded = padding_strategy != PaddingStrategy.DO_NOT_PAD

      # Initialize attention mask if not present.
      if return_attention_mask and "attention_mask" not in encoded_inputs:
          encoded_inputs["attention_mask"] = [1] * len(required_input)

      if needs_to_be_padded:
          sen_difference = max_sen_length - len(required_input)

          if self.padding_side == "right":
              if return_attention_mask:
                  encoded_inputs["sen_attention_mask"] = encoded_inputs["sen_attention_mask"] + [0] * sen_difference
              if "sen_token_type_ids" in encoded_inputs:
                  encoded_inputs["sen_token_type_ids"] = (
                      encoded_inputs["sen_token_type_ids"] + [self.pad_token_type_id] * sen_difference
                  )
              for i, input_ids in enumerate(encoded_inputs['input_ids']):
                difference = max_length - len(input_ids)
                if difference > 0:
                  encoded_inputs['input_ids'][i] = np.append(encoded_inputs['input_ids'][i], [0] * difference)
                  #encoded_inputs['token_type_ids'][i] = np.append(encoded_inputs['token_type_ids'][i], [self.pad_token_type_id] * difference)
                  encoded_inputs['attention_mask'][i] = np.append(encoded_inputs['attention_mask'][i], [0] * difference)
                  if "special_tokens_mask" in encoded_inputs:
                    encoded_inputs["special_tokens_mask"][i] = np.append(encoded_inputs["special_tokens_mask"][i], [1] * difference)
              if sen_difference > 0:
                encoded_inputs['input_ids'] = encoded_inputs['input_ids'] + [[0] * max_length] * sen_difference
                #encoded_inputs['token_type_ids'] = encoded_inputs['token_type_ids'] + [[self.pad_token_type_id] * max_length] * sen_difference
                encoded_inputs['attention_mask'] = encoded_inputs['attention_mask'] + [[0] * max_length] * sen_difference
                encoded_inputs["special_tokens_mask"] = encoded_inputs["special_tokens_mask"] + [[1] * max_length] * sen_difference
          elif self.padding_side == "left":
              if return_attention_mask:
                  encoded_inputs["attention_mask"] = [0] * sen_difference + encoded_inputs["attention_mask"]
              if "token_type_ids" in encoded_inputs:
                  encoded_inputs["token_type_ids"] = [self.pad_token_type_id] * sen_difference + encoded_inputs[
                      "token_type_ids"
                  ]
              if "special_tokens_mask" in encoded_inputs:
                  encoded_inputs["special_tokens_mask"] = [1] * sen_difference + encoded_inputs["special_tokens_mask"]
              encoded_inputs[self.model_input_names[0]] = [self.pad_token_id] * sen_difference + required_input
          else:
              raise ValueError("Invalid padding strategy:" + str(self.padding_side))

      return encoded_inputs

In [ ]:
my_tokenizer = LNLFBertTokenizer.from_pretrained('roberta-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'LNLFBertTokenizer'.


In [ ]:
#my_tokenizer = SeqTokenizer(tokenizer)

In [ ]:
a = my_tokenizer({1: ["This is a first header", "This is second [MASK] header"],0: ["This is a first documents.", "This is a second documents. This is a second sentence."]},
                 padding=False, return_tensors=None, multi_texts=True, order=[1,0])

In [ ]:
text_column_name = 'text'
def tokenize_function(examples):
    return my_tokenizer(examples[text_column_name], multi_texts=False, truncation=True, max_length=64, max_sen_num=60, return_special_tokens_mask=True)

In [ ]:
#raw_datasets["train"][3]

In [ ]:
#tokenize_function(raw_datasets["train"][3])

In [ ]:
import random
import warnings
from collections.abc import Mapping
from dataclasses import dataclass
from random import randint
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

import numpy as np
from transformers.data.data_collator import DataCollatorMixin, _tf_collate_batch, _torch_collate_batch, _numpy_collate_batch
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

In [ ]:
@dataclass
class SeqDataCollatorForLanguageModeling(DataCollatorMixin):
    """
    Data collator used for language modeling. Inputs are dynamically padded to the maximum length of a batch if they
    are not all of the same length.

    Args:
        tokenizer ([`PreTrainedTokenizer`] or [`PreTrainedTokenizerFast`]):
            The tokenizer used for encoding the data.
        mlm (`bool`, *optional*, defaults to `True`):
            Whether or not to use masked language modeling. If set to `False`, the labels are the same as the inputs
            with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for non-masked
            tokens and the value to predict for the masked token.
        mlm_probability (`float`, *optional*, defaults to 0.15):
            The probability with which to (randomly) mask tokens in the input, when `mlm` is set to `True`.
        pad_to_multiple_of (`int`, *optional*):
            If set will pad the sequence to a multiple of the provided value.
        return_tensors (`str`):
            The type of Tensor to return. Allowable values are "np", "pt" and "tf".

    <Tip>

    For best performance, this data collator should be used with a dataset having items that are dictionaries or
    BatchEncoding, with the `"special_tokens_mask"` key, as returned by a [`PreTrainedTokenizer`] or a
    [`PreTrainedTokenizerFast`] with the argument `return_special_tokens_mask=True`.

    </Tip>"""

    tokenizer: PreTrainedTokenizerBase
    mlm: bool = True
    mlm_probability: float = 0.15
    pad_to_multiple_of: Optional[int] = None
    tf_experimental_compile: bool = False
    return_tensors: str = "pt"

    def __post_init__(self):
        if self.mlm and self.tokenizer.mask_token is None:
            raise ValueError(
                "This tokenizer does not have a mask token which is necessary for masked language modeling. "
                "You should pass `mlm=False` to train on causal language modeling instead."
            )
        if self.tf_experimental_compile:
            import tensorflow as tf

            self.tf_mask_tokens = tf.function(self.tf_mask_tokens, jit_compile=True)

    @staticmethod
    def tf_bernoulli(shape, probability):
        import tensorflow as tf

        prob_matrix = tf.fill(shape, probability)
        return tf.cast(prob_matrix - tf.random.uniform(shape, 0, 1) >= 0, tf.bool)

    def tf_mask_tokens(
        self, inputs: Any, vocab_size, mask_token_id, special_tokens_mask: Optional[Any] = None
    ) -> Tuple[Any, Any]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        import tensorflow as tf

        mask_token_id = tf.cast(mask_token_id, inputs.dtype)

        input_shape = tf.shape(inputs)
        # 1 for a special token, 0 for a normal token in the special tokens mask
        # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
        masked_indices = self.tf_bernoulli(input_shape, self.mlm_probability) & ~special_tokens_mask

        for idx in range(len(masked_indices)):
            if not tf.reduce_any(masked_indices[idx]):
                f_token = tf.cast(tf.one_hot(tf.fill((input_shape[0],), 1), depth=input_shape[-1]), dtype=tf.bool)
                masked_indices = masked_indices | f_token
                break
        # Replace unmasked indices with -100 in the labels since we only compute loss on masked tokens
        labels = tf.where(masked_indices, inputs, -100)

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = self.tf_bernoulli(input_shape, 0.8) & masked_indices

        inputs = tf.where(indices_replaced, mask_token_id, inputs)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = self.tf_bernoulli(input_shape, 0.1) & masked_indices & ~indices_replaced
        random_words = tf.random.uniform(input_shape, maxval=vocab_size, dtype=inputs.dtype)

        inputs = tf.where(indices_random, random_words, inputs)

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels

    def tf_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        #print("examples:", examples)
        import tensorflow as tf

        # Handle dict or lists with proper padding and conversion to tensor.
        if isinstance(examples[0], Mapping):
            batch = self.tokenizer.pad(examples, return_tensors="tf", pad_to_multiple_of=self.pad_to_multiple_of)
        else:
            batch = {
                "input_ids": _tf_collate_batch(examples, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
            }

        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        if self.mlm:
            #print("special_tokens_mask:", special_tokens_mask)
            if special_tokens_mask is None:
                input_ids = batch["input_ids"].numpy()
                input_shape = np.shape(input_ids)
                flat_input = np.reshape(input_ids, (-1))
                special_tokens_mask = np.array(self.tokenizer.get_special_tokens_mask(flat_input, already_has_special_tokens=True))
                special_tokens_mask = np.reshape(special_tokens_mask, input_shape)
                """special_tokens_mask = [
                    self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
                    for val in batch["input_ids"].numpy().tolist()
                ]
                """
                # Cannot directly create as bool
                special_tokens_mask = tf.cast(tf.convert_to_tensor(special_tokens_mask, dtype=tf.int64), tf.bool)
            else:
                special_tokens_mask = tf.cast(special_tokens_mask, tf.bool)
            batch["input_ids"], batch["labels"] = self.tf_mask_tokens(
                tf.cast(batch["input_ids"], tf.int64),
                special_tokens_mask=special_tokens_mask,
                mask_token_id=self.tokenizer.mask_token_id,
                vocab_size=len(self.tokenizer),
            )
        else:
            labels = batch["input_ids"]
            if self.tokenizer.pad_token_id is not None:
                # Replace self.tokenizer.pad_token_id with -100
                labels = tf.where(labels == self.tokenizer.pad_token_id, -100, labels)
            else:
                labels = tf.identity(labels)  # Makes a copy, just in case
            batch["labels"] = labels
        return batch

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        # Handle dict or lists with proper padding and conversion to tensor.
        if isinstance(examples[0], Mapping):
            batch = self.tokenizer.pad(examples, return_tensors="pt", pad_to_multiple_of=self.pad_to_multiple_of)
        else:
            batch = {
                "input_ids": _torch_collate_batch(examples, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
            }

        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        if self.mlm:
            batch["input_ids"], batch["labels"] = self.torch_mask_tokens(
                batch["input_ids"], special_tokens_mask=special_tokens_mask
            )
        else:
            labels = batch["input_ids"].clone()
            if self.tokenizer.pad_token_id is not None:
                labels[labels == self.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
        return batch

    def torch_mask_tokens(self, inputs: Any, special_tokens_mask: Optional[Any] = None) -> Tuple[Any, Any]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        import torch

        labels = inputs.clone()
        # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
        probability_matrix = torch.full(labels.shape, self.mlm_probability)
        if special_tokens_mask is None:
            special_tokens_mask = [
                self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
            ]
            special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool)
        else:
            special_tokens_mask = special_tokens_mask.bool()

        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels

    def numpy_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        # Handle dict or lists with proper padding and conversion to tensor.
        if isinstance(examples[0], Mapping):
            batch = self.tokenizer.pad(examples, return_tensors="np", pad_to_multiple_of=self.pad_to_multiple_of)
        else:
            batch = {
                "input_ids": _numpy_collate_batch(examples, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
            }

        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        if self.mlm:
            batch["input_ids"], batch["labels"] = self.numpy_mask_tokens(
                batch["input_ids"], special_tokens_mask=special_tokens_mask
            )
        else:
            labels = np.copy(batch["input_ids"])
            if self.tokenizer.pad_token_id is not None:
                labels[labels == self.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
        return batch

    def numpy_mask_tokens(self, inputs: Any, special_tokens_mask: Optional[Any] = None) -> Tuple[Any, Any]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        labels = np.copy(inputs)
        # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
        probability_matrix = np.full(labels.shape, self.mlm_probability)
        if special_tokens_mask is None:
            special_tokens_mask = [
                self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
            ]
            special_tokens_mask = np.array(special_tokens_mask, dtype=bool)
        else:
            special_tokens_mask = special_tokens_mask.astype(bool)

        probability_matrix[special_tokens_mask] = 0
        # Numpy doesn't have bernoulli, so we use a binomial with 1 trial
        masked_indices = np.random.binomial(1, probability_matrix, size=probability_matrix.shape).astype(bool)
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = np.random.binomial(1, 0.8, size=labels.shape).astype(bool) & masked_indices
        inputs[indices_replaced] = self.tokenizer.mask_token_id

        # 10% of the time, we replace masked input tokens with random word
        # indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        indices_random = (
            np.random.binomial(1, 0.5, size=labels.shape).astype(bool) & masked_indices & ~indices_replaced
        )
        random_words = np.random.randint(
            low=0, high=len(self.tokenizer), size=np.count_nonzero(indices_random), dtype=np.int64
        )
        inputs[indices_random] = random_words

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels

In [ ]:
mlm_probability = 0.15
data_collator = SeqDataCollatorForLanguageModeling(
    tokenizer=my_tokenizer, mlm_probability=mlm_probability, return_tensors="tf"
)

In [ ]:
dataset_name = 'wikitext'
dataset_config_name = 'wikitext-103-raw-v1'
raw_datasets = load_dataset(
    dataset_name,
    dataset_config_name,
    token=None,
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=None,
    remove_columns=[text_column_name],
    load_from_cache_file=True,
    desc="Running tokenizer on dataset line_by_line",
)
tokenized_datasets = tokenized_datasets.filter(lambda example: len(example['input_ids']) > 0)

Running tokenizer on dataset line_by_line:   0%|          | 0/4358 [00:00<?, ? examples/s]

Running tokenizer on dataset line_by_line:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Running tokenizer on dataset line_by_line:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets['test']
valid_dataset = tokenized_datasets["validation"]

In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

In [ ]:
initial_epoch = 168

In [ ]:
#config.num_labels=2
model_pre_path = "./drive/MyDrive/Models/TFLNLFBert_Pretrain_from_scratch/epoch"
if initial_epoch <= 0:
  new_model = TFLNLFBertForMaskedLM.from_config(config)
  new_model.build()
else:
  new_model = TFLNLFBertForMaskedLM.from_pretrained(model_pre_path + str(initial_epoch - 1))

All model checkpoint layers were used when initializing TFLNLFBertForMaskedLM.

All the layers of TFLNLFBertForMaskedLM were initialized from the model checkpoint at ./drive/MyDrive/Models/TFLNLFBert_Pretrain_from_scratch/epoch167.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLNLFBertForMaskedLM for predictions without further training.


In [ ]:
tf_train_dataset = new_model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
    drop_remainder=True,
).with_options(options)

In [ ]:
tf_valid_dataset = new_model.prepare_tf_dataset(
    valid_dataset,
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator,
    drop_remainder=True,
).with_options(options)

In [ ]:
tf_test_dataset = new_model.prepare_tf_dataset(
    test_dataset,
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator,
).with_options(options)

In [ ]:
num_train_epochs = 3
base_learning_rate=5e-5
num_warmup_steps = 0
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 1e-8
weight_decay = 0.0
max_grad_norm = 1.0
xla=False

In [ ]:
split_time = 60
epochs = int(num_train_epochs * split_time)
steps_per_epoch = int(len(tf_train_dataset) // split_time)
num_epochs = 30
next_train_epochs = min(initial_epoch + num_epochs, epochs)

In [ ]:
base_train_steps = len(tf_train_dataset) * int(num_train_epochs)

In [ ]:
num_train_steps = base_train_steps - initial_epoch * steps_per_epoch

In [ ]:
learning_rate = base_learning_rate * num_train_steps / base_train_steps

In [ ]:
print(learning_rate)

3.3336938393808224e-06


In [ ]:
from transformers import create_optimizer

In [ ]:
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    adam_beta1=adam_beta1,
    adam_beta2=adam_beta2,
    adam_epsilon=adam_epsilon,
    weight_decay_rate=weight_decay,
    adam_global_clipnorm=max_grad_norm,
)

In [ ]:
new_model.compile(optimizer=optimizer, jit_compile=xla)

In [ ]:
import tensorflow.keras.backend as K
print(K.eval(new_model.optimizer.lr))

3.333694e-06


In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start save model epoch ", epoch)
        self.model.save_pretrained(model_pre_path + str(epoch))
        print("Optimizer learning rate = ", K.eval(self.model.optimizer.lr))
        #print("Optimizer learning rate = ", self.model.optimizer._decayed_lr(tf.float32))
        print("End save model epoch ", epoch)

In [ ]:
callbacks = [CustomCallback()]

In [ ]:
new_model.summary()

Model: "tflnlf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lnlfbert (TFLNLFBertMainLa  multiple                  202078464 
 yer)                                                            
                                                                 
 lm_head (TFLNLFBertLMHead)  multiple                  39642969  
                                                                 
Total params: 202720857 (773.32 MB)
Trainable params: 202720857 (773.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = new_model.fit(
    tf_train_dataset,
    validation_data=tf_valid_dataset,
    epochs=next_train_epochs,
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    callbacks=callbacks,
)

Epoch 169/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4611Start save model epoch  168
Optimizer learning rate =  3.0559327e-06
End save model epoch  168
19417/19417 [==============================] - 3877s 195ms/step - loss: 2.4611 - val_loss: 2.4056
Epoch 170/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4573Start save model epoch  169
Optimizer learning rate =  2.7781568e-06
End save model epoch  169
19417/19417 [==============================] - 3748s 193ms/step - loss: 2.4573 - val_loss: 2.4147
Epoch 171/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4252Start save model epoch  170
Optimizer learning rate =  2.5003812e-06
End save model epoch  170
19417/19417 [==============================] - 3730s 192ms/step - loss: 2.4252 - val_loss: 2.4072
Epoch 172/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4164Start save model epoch  171
Optimizer learning rate =  2.2226056e-06
End save model epoch

In [ ]:
"""
Epoch 1/30
19417/19417 [==============================] - ETA: 0s - loss: 5.6922Start save model epoch  0
Optimizer learning rate =  4.972224e-05
End save model epoch  0
19417/19417 [==============================] - 3936s 198ms/step - loss: 5.6922 - val_loss: 5.2161
Epoch 2/30
19417/19417 [==============================] - ETA: 0s - loss: 4.8180Start save model epoch  1
Optimizer learning rate =  4.944446e-05
End save model epoch  1
19417/19417 [==============================] - 3830s 197ms/step - loss: 4.8180 - val_loss: 4.6530
Epoch 3/30
19417/19417 [==============================] - ETA: 0s - loss: 4.5446Start save model epoch  2
Optimizer learning rate =  4.9166687e-05
End save model epoch  2
19417/19417 [==============================] - 3808s 196ms/step - loss: 4.5446 - val_loss: 4.3152
Epoch 4/30
19417/19417 [==============================] - ETA: 0s - loss: 4.2730Start save model epoch  3
Optimizer learning rate =  4.8888913e-05
End save model epoch  3
19417/19417 [==============================] - 3729s 192ms/step - loss: 4.2730 - val_loss: 4.1824
Epoch 5/30
19417/19417 [==============================] - ETA: 0s - loss: 4.2106Start save model epoch  4
Optimizer learning rate =  4.8611135e-05
End save model epoch  4
19417/19417 [==============================] - 3791s 195ms/step - loss: 4.2106 - val_loss: 4.0386
Epoch 6/30
19417/19417 [==============================] - ETA: 0s - loss: 4.1498Start save model epoch  5
Optimizer learning rate =  4.833336e-05
End save model epoch  5
19417/19417 [==============================] - 3840s 198ms/step - loss: 4.1498 - val_loss: 4.0779
Epoch 7/30
19417/19417 [==============================] - ETA: 0s - loss: 4.0457Start save model epoch  6
Optimizer learning rate =  4.8055583e-05
End save model epoch  6
19417/19417 [==============================] - 3791s 195ms/step - loss: 4.0457 - val_loss: 3.9875
Epoch 8/30
19417/19417 [==============================] - ETA: 0s - loss: 3.9871Start save model epoch  7
Optimizer learning rate =  4.7777805e-05
End save model epoch  7
19417/19417 [==============================] - 3745s 193ms/step - loss: 3.9871 - val_loss: 3.8995
Epoch 9/30
19417/19417 [==============================] - ETA: 0s - loss: 3.9782Start save model epoch  8
Optimizer learning rate =  4.750003e-05
End save model epoch  8
19417/19417 [==============================] - 3833s 197ms/step - loss: 3.9782 - val_loss: 3.8293
Epoch 10/30
19417/19417 [==============================] - ETA: 0s - loss: 3.8852Start save model epoch  9
Optimizer learning rate =  4.7222256e-05
End save model epoch  9
19417/19417 [==============================] - 3738s 192ms/step - loss: 3.8852 - val_loss: 3.8870
Epoch 11/30
19417/19417 [==============================] - ETA: 0s - loss: 3.8409Start save model epoch  10
Optimizer learning rate =  4.694448e-05
End save model epoch  10
19417/19417 [==============================] - 3829s 197ms/step - loss: 3.8409 - val_loss: 3.7197
Epoch 12/30
19417/19417 [==============================] - ETA: 0s - loss: 3.8255Start save model epoch  11
Optimizer learning rate =  4.6666704e-05
End save model epoch  11
19417/19417 [==============================] - 3839s 198ms/step - loss: 3.8255 - val_loss: 3.7656
Epoch 13/30
19417/19417 [==============================] - ETA: 0s - loss: 3.8343Start save model epoch  12
Optimizer learning rate =  4.638893e-05
End save model epoch  12
19417/19417 [==============================] - 3780s 195ms/step - loss: 3.8343 - val_loss: 3.6481
Epoch 14/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7529Start save model epoch  13
Optimizer learning rate =  4.6111152e-05
End save model epoch  13
19417/19417 [==============================] - 3814s 196ms/step - loss: 3.7529 - val_loss: 3.6738
Epoch 15/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7644Start save model epoch  14
Optimizer learning rate =  4.5833378e-05
End save model epoch  14
19417/19417 [==============================] - 3837s 198ms/step - loss: 3.7644 - val_loss: 3.7138
Epoch 16/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7273Start save model epoch  15
Optimizer learning rate =  4.5555604e-05
End save model epoch  15
19417/19417 [==============================] - 3862s 199ms/step - loss: 3.7273 - val_loss: 3.5868
Epoch 17/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7104Start save model epoch  16
Optimizer learning rate =  4.5277826e-05
End save model epoch  16
19417/19417 [==============================] - 3830s 197ms/step - loss: 3.7104 - val_loss: 3.6098
Epoch 18/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7376Start save model epoch  17
Optimizer learning rate =  4.500005e-05
End save model epoch  17
19417/19417 [==============================] - 3848s 198ms/step - loss: 3.7376 - val_loss: 3.6834
Epoch 19/30
19417/19417 [==============================] - ETA: 0s - loss: 3.7163Start save model epoch  18
Optimizer learning rate =  4.4722277e-05
End save model epoch  18
19417/19417 [==============================] - 3754s 193ms/step - loss: 3.7163 - val_loss: 3.6179
Epoch 20/30
19417/19417 [==============================] - ETA: 0s - loss: 3.6676Start save model epoch  19
Optimizer learning rate =  4.44445e-05
End save model epoch  19
19417/19417 [==============================] - 3824s 197ms/step - loss: 3.6676 - val_loss: 3.6925
Epoch 21/30
19417/19417 [==============================] - ETA: 0s - loss: 3.6438Start save model epoch  20
Optimizer learning rate =  4.4166725e-05
End save model epoch  20
19417/19417 [==============================] - 3869s 199ms/step - loss: 3.6438 - val_loss: 3.5622
Epoch 22/30
19110/19417 [============================>.] - ETA: 57s - loss: 3.6374



Epoch 22/51
19417/19417 [==============================] - ETA: 0s - loss: 3.7446Start save model epoch  21
Optimizer learning rate =  4.3888947e-05
End save model epoch  21
19417/19417 [==============================] - 3829s 192ms/step - loss: 3.7446 - val_loss: 3.6213
Epoch 23/51
19417/19417 [==============================] - ETA: 0s - loss: 3.6621Start save model epoch  22
Optimizer learning rate =  4.3611173e-05
End save model epoch  22
19417/19417 [==============================] - 3745s 193ms/step - loss: 3.6621 - val_loss: 3.5728
Epoch 24/51
19417/19417 [==============================] - ETA: 0s - loss: 3.6345Start save model epoch  23
Optimizer learning rate =  4.33334e-05
End save model epoch  23
19417/19417 [==============================] - 3800s 196ms/step - loss: 3.6345 - val_loss: 3.5487
Epoch 25/51
19417/19417 [==============================] - ETA: 0s - loss: 3.6015Start save model epoch  24
Optimizer learning rate =  4.3055625e-05
End save model epoch  24
19417/19417 [==============================] - 3802s 196ms/step - loss: 3.6015 - val_loss: 3.4806
Epoch 26/51
19417/19417 [==============================] - ETA: 0s - loss: 3.6191Start save model epoch  25
Optimizer learning rate =  4.2777847e-05
End save model epoch  25
19417/19417 [==============================] - 3784s 195ms/step - loss: 3.6191 - val_loss: 3.4558
Epoch 27/51
19417/19417 [==============================] - ETA: 0s - loss: 3.5649Start save model epoch  26
Optimizer learning rate =  4.250007e-05
End save model epoch  26
19417/19417 [==============================] - 3702s 191ms/step - loss: 3.5649 - val_loss: 3.3910
Epoch 28/51
19417/19417 [==============================] - ETA: 0s - loss: 3.5401Start save model epoch  27
Optimizer learning rate =  4.2222295e-05
End save model epoch  27
19417/19417 [==============================] - 3710s 191ms/step - loss: 3.5401 - val_loss: 3.4727
Epoch 29/51
19417/19417 [==============================] - ETA: 0s - loss: 3.5013Start save model epoch  28
Optimizer learning rate =  4.194452e-05
End save model epoch  28
19417/19417 [==============================] - 3751s 193ms/step - loss: 3.5013 - val_loss: 3.4204
Epoch 30/51
19417/19417 [==============================] - ETA: 0s - loss: 3.5032Start save model epoch  29
Optimizer learning rate =  4.1666743e-05
End save model epoch  29
19417/19417 [==============================] - 3754s 193ms/step - loss: 3.5032 - val_loss: 3.3860
Epoch 31/51
19417/19417 [==============================] - ETA: 0s - loss: 3.4825Start save model epoch  30
Optimizer learning rate =  4.138897e-05
End save model epoch  30
19417/19417 [==============================] - 3816s 197ms/step - loss: 3.4825 - val_loss: 3.4034
Epoch 32/51
19417/19417 [==============================] - ETA: 0s - loss: 3.4346Start save model epoch  31
Optimizer learning rate =  4.1111194e-05
End save model epoch  31
19417/19417 [==============================] - 3782s 195ms/step - loss: 3.4346 - val_loss: 3.3916
Epoch 33/51
19417/19417 [==============================] - ETA: 0s - loss: 3.4406Start save model epoch  32
Optimizer learning rate =  4.083342e-05
End save model epoch  32
19417/19417 [==============================] - 3798s 196ms/step - loss: 3.4406 - val_loss: 3.3293
Epoch 34/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3830Start save model epoch  33
Optimizer learning rate =  4.0555642e-05
End save model epoch  33
19417/19417 [==============================] - 3754s 193ms/step - loss: 3.3830 - val_loss: 3.2969
Epoch 35/51
19417/19417 [==============================] - ETA: 0s - loss: 3.4111Start save model epoch  34
Optimizer learning rate =  4.0277864e-05
End save model epoch  34
19417/19417 [==============================] - 3797s 196ms/step - loss: 3.4111 - val_loss: 3.3084
Epoch 36/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3829Start save model epoch  35
Optimizer learning rate =  4.000009e-05
End save model epoch  35
19417/19417 [==============================] - 3846s 198ms/step - loss: 3.3829 - val_loss: 3.2943
Epoch 37/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3647Start save model epoch  36
Optimizer learning rate =  3.9722316e-05
End save model epoch  36
19417/19417 [==============================] - 3851s 198ms/step - loss: 3.3647 - val_loss: 3.3300
Epoch 38/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3694Start save model epoch  37
Optimizer learning rate =  3.944454e-05
End save model epoch  37
19417/19417 [==============================] - 3795s 195ms/step - loss: 3.3694 - val_loss: 3.2008
Epoch 39/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3586Start save model epoch  38
Optimizer learning rate =  3.9166764e-05
End save model epoch  38
19417/19417 [==============================] - 3884s 200ms/step - loss: 3.3586 - val_loss: 3.2105
Epoch 40/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3728Start save model epoch  39
Optimizer learning rate =  3.8888986e-05
End save model epoch  39
19417/19417 [==============================] - 3817s 197ms/step - loss: 3.3728 - val_loss: 3.2700
Epoch 41/51
19417/19417 [==============================] - ETA: 0s - loss: 3.2994Start save model epoch  40
Optimizer learning rate =  3.861121e-05
End save model epoch  40
19417/19417 [==============================] - 3788s 195ms/step - loss: 3.2994 - val_loss: 3.2454
Epoch 42/51
19417/19417 [==============================] - ETA: 0s - loss: 3.3345Start save model epoch  41
Optimizer learning rate =  3.8333437e-05
End save model epoch  41
19417/19417 [==============================] - 3778s 195ms/step - loss: 3.3345 - val_loss: 3.1783
Epoch 43/51
  694/19417 [>.............................] - ETA: 59:29 - loss: 3.4326

Epoch 43/72
19417/19417 [==============================] - ETA: 0s - loss: 3.3956Start save model epoch  42
Optimizer learning rate =  3.805566e-05
End save model epoch  42
19417/19417 [==============================] - 4167s 209ms/step - loss: 3.3956 - val_loss: 3.2789
Epoch 44/72
19417/19417 [==============================] - ETA: 0s - loss: 3.3323Start save model epoch  43
Optimizer learning rate =  3.7777885e-05
End save model epoch  43
19417/19417 [==============================] - 4012s 207ms/step - loss: 3.3323 - val_loss: 3.2746
Epoch 45/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2965Start save model epoch  44
Optimizer learning rate =  3.750011e-05
End save model epoch  44
19417/19417 [==============================] - 4043s 208ms/step - loss: 3.2965 - val_loss: 3.2737
Epoch 46/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2611Start save model epoch  45
Optimizer learning rate =  3.7222333e-05
End save model epoch  45
19417/19417 [==============================] - 4018s 207ms/step - loss: 3.2611 - val_loss: 3.1583
Epoch 47/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2878Start save model epoch  46
Optimizer learning rate =  3.694456e-05
End save model epoch  46
19417/19417 [==============================] - 3994s 206ms/step - loss: 3.2878 - val_loss: 3.1686
Epoch 48/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2595Start save model epoch  47
Optimizer learning rate =  3.666678e-05
End save model epoch  47
19417/19417 [==============================] - 3994s 206ms/step - loss: 3.2595 - val_loss: 3.1826
Epoch 49/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2148Start save model epoch  48
Optimizer learning rate =  3.6389007e-05
End save model epoch  48
19417/19417 [==============================] - 3998s 206ms/step - loss: 3.2148 - val_loss: 3.1843
Epoch 50/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2061Start save model epoch  49
Optimizer learning rate =  3.6111232e-05
End save model epoch  49
19417/19417 [==============================] - 3993s 206ms/step - loss: 3.2061 - val_loss: 3.2404
Epoch 51/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2656Start save model epoch  50
Optimizer learning rate =  3.5833455e-05
End save model epoch  50
19417/19417 [==============================] - 3994s 206ms/step - loss: 3.2656 - val_loss: 3.0325
Epoch 52/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2231Start save model epoch  51
Optimizer learning rate =  3.555568e-05
End save model epoch  51
19417/19417 [==============================] - 3988s 205ms/step - loss: 3.2231 - val_loss: 3.0876
Epoch 53/72
19417/19417 [==============================] - ETA: 0s - loss: 3.2183Start save model epoch  52
Optimizer learning rate =  3.5277902e-05
End save model epoch  52
19417/19417 [==============================] - 4003s 206ms/step - loss: 3.2183 - val_loss: 3.1248
Epoch 54/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1414Start save model epoch  53
Optimizer learning rate =  3.5000132e-05
End save model epoch  53
19417/19417 [==============================] - 3999s 206ms/step - loss: 3.1414 - val_loss: 3.1112
Epoch 55/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1755Start save model epoch  54
Optimizer learning rate =  3.4722354e-05
End save model epoch  54
19417/19417 [==============================] - 3994s 206ms/step - loss: 3.1755 - val_loss: 3.1104
Epoch 56/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1547Start save model epoch  55
Optimizer learning rate =  3.444458e-05
End save model epoch  55
19417/19417 [==============================] - 4084s 210ms/step - loss: 3.1547 - val_loss: 3.0524
Epoch 57/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1595Start save model epoch  56
Optimizer learning rate =  3.4166802e-05
End save model epoch  56
19417/19417 [==============================] - 4024s 207ms/step - loss: 3.1595 - val_loss: 3.1124
Epoch 58/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1837Start save model epoch  57
Optimizer learning rate =  3.3889028e-05
End save model epoch  57
19417/19417 [==============================] - 4048s 208ms/step - loss: 3.1837 - val_loss: 3.0796
Epoch 59/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1408Start save model epoch  58
Optimizer learning rate =  3.3611253e-05
End save model epoch  58
19417/19417 [==============================] - 4051s 209ms/step - loss: 3.1408 - val_loss: 3.1243
Epoch 60/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1329Start save model epoch  59
Optimizer learning rate =  3.3333476e-05
End save model epoch  59
19417/19417 [==============================] - 4030s 208ms/step - loss: 3.1329 - val_loss: 2.9879
Epoch 61/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1329Start save model epoch  60
Optimizer learning rate =  3.30557e-05
End save model epoch  60
19417/19417 [==============================] - 3981s 205ms/step - loss: 3.1329 - val_loss: 3.0376
Epoch 62/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1546Start save model epoch  61
Optimizer learning rate =  3.2777923e-05
End save model epoch  61
19417/19417 [==============================] - 4033s 208ms/step - loss: 3.1546 - val_loss: 2.9813
Epoch 63/72
19417/19417 [==============================] - ETA: 0s - loss: 3.1302Start save model epoch  62
Optimizer learning rate =  3.250015e-05
End save model epoch  62
19417/19417 [==============================] - 4020s 207ms/step - loss: 3.1302 - val_loss: 2.9018
Epoch 64/72
 3600/19417 [====>.........................] - ETA: 52:50 - loss: 3.1251

Epoch 64/93
19417/19417 [==============================] - ETA: 0s - loss: 3.2231Start save model epoch  63
Optimizer learning rate =  3.2222375e-05
End save model epoch  63
19417/19417 [==============================] - 3934s 198ms/step - loss: 3.2231 - val_loss: 3.0977
Epoch 65/93
19417/19417 [==============================] - ETA: 0s - loss: 3.1367Start save model epoch  64
Optimizer learning rate =  3.1944597e-05
End save model epoch  64
19417/19417 [==============================] - 3828s 197ms/step - loss: 3.1367 - val_loss: 3.0479
Epoch 66/93
19417/19417 [==============================] - ETA: 0s - loss: 3.1119Start save model epoch  65
Optimizer learning rate =  3.1666823e-05
End save model epoch  65
19417/19417 [==============================] - 3803s 196ms/step - loss: 3.1119 - val_loss: 3.0071
Epoch 67/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0642Start save model epoch  66
Optimizer learning rate =  3.1389045e-05
End save model epoch  66
19417/19417 [==============================] - 3773s 194ms/step - loss: 3.0642 - val_loss: 2.9904
Epoch 68/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0735Start save model epoch  67
Optimizer learning rate =  3.111127e-05
End save model epoch  67
19417/19417 [==============================] - 3728s 192ms/step - loss: 3.0735 - val_loss: 2.9921
Epoch 69/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0711Start save model epoch  68
Optimizer learning rate =  3.0833497e-05
End save model epoch  68
19417/19417 [==============================] - 3781s 195ms/step - loss: 3.0711 - val_loss: 2.9452
Epoch 70/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0389Start save model epoch  69
Optimizer learning rate =  3.055572e-05
End save model epoch  69
19417/19417 [==============================] - 3752s 193ms/step - loss: 3.0389 - val_loss: 3.0024
Epoch 71/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0037Start save model epoch  70
Optimizer learning rate =  3.0277944e-05
End save model epoch  70
19417/19417 [==============================] - 3754s 193ms/step - loss: 3.0037 - val_loss: 2.9549
Epoch 72/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0457Start save model epoch  71
Optimizer learning rate =  3.0000168e-05
End save model epoch  71
19417/19417 [==============================] - 3748s 193ms/step - loss: 3.0457 - val_loss: 2.9619
Epoch 73/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0322Start save model epoch  72
Optimizer learning rate =  2.9722392e-05
End save model epoch  72
19417/19417 [==============================] - 3759s 194ms/step - loss: 3.0322 - val_loss: 3.0170
Epoch 74/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0077Start save model epoch  73
Optimizer learning rate =  2.9444616e-05
End save model epoch  73
19417/19417 [==============================] - 3804s 196ms/step - loss: 3.0077 - val_loss: 2.9991
Epoch 75/93
19417/19417 [==============================] - ETA: 0s - loss: 2.9987Start save model epoch  74
Optimizer learning rate =  2.9166842e-05
End save model epoch  74
19417/19417 [==============================] - 3847s 198ms/step - loss: 2.9987 - val_loss: 3.0218
Epoch 76/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0278Start save model epoch  75
Optimizer learning rate =  2.8889064e-05
End save model epoch  75
19417/19417 [==============================] - 3780s 195ms/step - loss: 3.0278 - val_loss: 2.9013
Epoch 77/93
19417/19417 [==============================] - ETA: 0s - loss: 3.0112Start save model epoch  76
Optimizer learning rate =  2.861129e-05
End save model epoch  76
19417/19417 [==============================] - 3775s 194ms/step - loss: 3.0112 - val_loss: 2.9667
Epoch 78/93
19417/19417 [==============================] - ETA: 0s - loss: 2.9643Start save model epoch  77
Optimizer learning rate =  2.8333516e-05
End save model epoch  77
19417/19417 [==============================] - 3779s 195ms/step - loss: 2.9643 - val_loss: 2.9515
Epoch 79/93
19417/19417 [==============================] - ETA: 0s - loss: 2.9768Start save model epoch  78
Optimizer learning rate =  2.8055738e-05
End save model epoch  78
19417/19417 [==============================] - 3726s 192ms/step - loss: 2.9768 - val_loss: 2.8969
Epoch 80/93
 3672/19417 [====>.........................] - ETA: 48:14 - loss: 3.0621

Epoch 80/109
19417/19417 [==============================] - ETA: 0s - loss: 3.0467Start save model epoch  79
Optimizer learning rate =  2.7777964e-05
End save model epoch  79
19417/19417 [==============================] - 3832s 192ms/step - loss: 3.0467 - val_loss: 2.9601
Epoch 81/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9642Start save model epoch  80
Optimizer learning rate =  2.7500188e-05
End save model epoch  80
19417/19417 [==============================] - 3732s 192ms/step - loss: 2.9642 - val_loss: 2.8721
Epoch 82/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9248Start save model epoch  81
Optimizer learning rate =  2.7222412e-05
End save model epoch  81
19417/19417 [==============================] - 3735s 192ms/step - loss: 2.9248 - val_loss: 2.8670
Epoch 83/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9237Start save model epoch  82
Optimizer learning rate =  2.6944637e-05
End save model epoch  82
19417/19417 [==============================] - 3691s 190ms/step - loss: 2.9237 - val_loss: 2.8790
Epoch 84/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9417Start save model epoch  83
Optimizer learning rate =  2.6666861e-05
End save model epoch  83
19417/19417 [==============================] - 3783s 195ms/step - loss: 2.9417 - val_loss: 2.9022
Epoch 85/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8973Start save model epoch  84
Optimizer learning rate =  2.6389085e-05
End save model epoch  84
19417/19417 [==============================] - 3703s 191ms/step - loss: 2.8973 - val_loss: 2.8604
Epoch 86/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9158Start save model epoch  85
Optimizer learning rate =  2.611131e-05
End save model epoch  85
19417/19417 [==============================] - 3757s 193ms/step - loss: 2.9158 - val_loss: 2.9124
Epoch 87/109
19417/19417 [==============================] - ETA: 0s - loss: 2.9148Start save model epoch  86
Optimizer learning rate =  2.5833533e-05
End save model epoch  86
19417/19417 [==============================] - 3768s 194ms/step - loss: 2.9148 - val_loss: 2.8472
Epoch 88/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8825Start save model epoch  87
Optimizer learning rate =  2.5555759e-05
End save model epoch  87
19417/19417 [==============================] - 3767s 194ms/step - loss: 2.8825 - val_loss: 2.8804
Epoch 89/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8965Start save model epoch  88
Optimizer learning rate =  2.5277983e-05
End save model epoch  88
19417/19417 [==============================] - 3762s 194ms/step - loss: 2.8965 - val_loss: 2.8711
Epoch 90/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8561Start save model epoch  89
Optimizer learning rate =  2.5000207e-05
End save model epoch  89
19417/19417 [==============================] - 3816s 197ms/step - loss: 2.8561 - val_loss: 2.7870
Epoch 91/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8834Start save model epoch  90
Optimizer learning rate =  2.4722432e-05
End save model epoch  90
19417/19417 [==============================] - 3806s 196ms/step - loss: 2.8834 - val_loss: 2.8637
Epoch 92/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8589Start save model epoch  91
Optimizer learning rate =  2.4444656e-05
End save model epoch  91
19417/19417 [==============================] - 3858s 199ms/step - loss: 2.8589 - val_loss: 2.8083
Epoch 93/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8601Start save model epoch  92
Optimizer learning rate =  2.416688e-05
End save model epoch  92
19417/19417 [==============================] - 3863s 199ms/step - loss: 2.8601 - val_loss: 2.7794
Epoch 94/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8724Start save model epoch  93
Optimizer learning rate =  2.3889104e-05
End save model epoch  93
19417/19417 [==============================] - 3834s 197ms/step - loss: 2.8724 - val_loss: 2.8316
Epoch 95/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8460Start save model epoch  94
Optimizer learning rate =  2.3611328e-05
End save model epoch  94
19417/19417 [==============================] - 3882s 200ms/step - loss: 2.8460 - val_loss: 2.7165
Epoch 96/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8210Start save model epoch  95
Optimizer learning rate =  2.3333554e-05
End save model epoch  95
19417/19417 [==============================] - 3825s 197ms/step - loss: 2.8210 - val_loss: 2.8142
Epoch 97/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8386Start save model epoch  96
Optimizer learning rate =  2.3055778e-05
End save model epoch  96
19417/19417 [==============================] - 3861s 199ms/step - loss: 2.8386 - val_loss: 2.8096
Epoch 98/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8302Start save model epoch  97
Optimizer learning rate =  2.2778002e-05
End save model epoch  97
19417/19417 [==============================] - 3855s 199ms/step - loss: 2.8302 - val_loss: 2.7780
Epoch 99/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8338Start save model epoch  98
Optimizer learning rate =  2.2500228e-05
End save model epoch  98
19417/19417 [==============================] - 3819s 197ms/step - loss: 2.8338 - val_loss: 2.7968
Epoch 100/109
19417/19417 [==============================] - ETA: 0s - loss: 2.8221Start save model epoch  99
Optimizer learning rate =  2.2222452e-05
End save model epoch  99
19417/19417 [==============================] - 3819s 197ms/step - loss: 2.8221 - val_loss: 2.8390
Epoch 101/109
19417/19417 [==============================] - ETA: 0s - loss: 2.7609Start save model epoch  100
Optimizer learning rate =  2.1944676e-05
End save model epoch  100
19417/19417 [==============================] - 3862s 199ms/step - loss: 2.7609 - val_loss: 2.6845
Epoch 102/109
 5698/19417 [=======>......................] - ETA: 43:03 - loss: 2.8216

Epoch 102/131
19417/19417 [==============================] - ETA: 0s - loss: 2.8678Start save model epoch  101
Optimizer learning rate =  2.16669e-05
End save model epoch  101
19417/19417 [==============================] - 3937s 198ms/step - loss: 2.8678 - val_loss: 2.7624
Epoch 103/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7787Start save model epoch  102
Optimizer learning rate =  2.1389124e-05
End save model epoch  102
19417/19417 [==============================] - 3748s 193ms/step - loss: 2.7787 - val_loss: 2.7331
Epoch 104/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7953Start save model epoch  103
Optimizer learning rate =  2.111135e-05
End save model epoch  103
19417/19417 [==============================] - 3804s 196ms/step - loss: 2.7953 - val_loss: 2.7340
Epoch 105/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7716Start save model epoch  104
Optimizer learning rate =  2.0833573e-05
End save model epoch  104
19417/19417 [==============================] - 3852s 198ms/step - loss: 2.7716 - val_loss: 2.7192
Epoch 106/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7510Start save model epoch  105
Optimizer learning rate =  2.0555797e-05
End save model epoch  105
19417/19417 [==============================] - 3785s 195ms/step - loss: 2.7510 - val_loss: 2.6898
Epoch 107/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7641Start save model epoch  106
Optimizer learning rate =  2.0278021e-05
End save model epoch  106
19417/19417 [==============================] - 3829s 197ms/step - loss: 2.7641 - val_loss: 2.7069
Epoch 108/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7486Start save model epoch  107
Optimizer learning rate =  2.0000245e-05
End save model epoch  107
19417/19417 [==============================] - 3737s 192ms/step - loss: 2.7486 - val_loss: 2.6922
Epoch 109/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7471Start save model epoch  108
Optimizer learning rate =  1.972247e-05
End save model epoch  108
19417/19417 [==============================] - 3730s 192ms/step - loss: 2.7471 - val_loss: 2.6689
Epoch 110/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7365Start save model epoch  109
Optimizer learning rate =  1.9444695e-05
End save model epoch  109
19417/19417 [==============================] - 3726s 192ms/step - loss: 2.7365 - val_loss: 2.6608
Epoch 111/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7351Start save model epoch  110
Optimizer learning rate =  1.9166919e-05
End save model epoch  110
19417/19417 [==============================] - 3688s 190ms/step - loss: 2.7351 - val_loss: 2.6985
Epoch 112/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7344Start save model epoch  111
Optimizer learning rate =  1.8889143e-05
End save model epoch  111
19417/19417 [==============================] - 3728s 192ms/step - loss: 2.7344 - val_loss: 2.6623
Epoch 113/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7199Start save model epoch  112
Optimizer learning rate =  1.8611368e-05
End save model epoch  112
19417/19417 [==============================] - 3736s 192ms/step - loss: 2.7199 - val_loss: 2.6550
Epoch 114/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7040Start save model epoch  113
Optimizer learning rate =  1.8333592e-05
End save model epoch  113
19417/19417 [==============================] - 3676s 189ms/step - loss: 2.7040 - val_loss: 2.6603
Epoch 115/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6928Start save model epoch  114
Optimizer learning rate =  1.8055816e-05
End save model epoch  114
19417/19417 [==============================] - 3766s 194ms/step - loss: 2.6928 - val_loss: 2.6333
Epoch 116/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6916Start save model epoch  115
Optimizer learning rate =  1.7778042e-05
End save model epoch  115
19417/19417 [==============================] - 3764s 194ms/step - loss: 2.6916 - val_loss: 2.6557
Epoch 117/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7029Start save model epoch  116
Optimizer learning rate =  1.7500266e-05
End save model epoch  116
19417/19417 [==============================] - 3795s 195ms/step - loss: 2.7029 - val_loss: 2.6414
Epoch 118/131
19417/19417 [==============================] - ETA: 0s - loss: 2.7168Start save model epoch  117
Optimizer learning rate =  1.722249e-05
End save model epoch  117
19417/19417 [==============================] - 3728s 192ms/step - loss: 2.7168 - val_loss: 2.6266
Epoch 119/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6727Start save model epoch  118
Optimizer learning rate =  1.6944714e-05
End save model epoch  118
19417/19417 [==============================] - 3746s 193ms/step - loss: 2.6727 - val_loss: 2.6598
Epoch 120/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6452Start save model epoch  119
Optimizer learning rate =  1.6666938e-05
End save model epoch  119
19417/19417 [==============================] - 3778s 195ms/step - loss: 2.6452 - val_loss: 2.6423
Epoch 121/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6696Start save model epoch  120
Optimizer learning rate =  1.6389162e-05
End save model epoch  120
19417/19417 [==============================] - 3787s 195ms/step - loss: 2.6696 - val_loss: 2.6314
Epoch 122/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6596Start save model epoch  121
Optimizer learning rate =  1.6111388e-05
End save model epoch  121
19417/19417 [==============================] - 3793s 195ms/step - loss: 2.6596 - val_loss: 2.5977
Epoch 123/131
19417/19417 [==============================] - ETA: 0s - loss: 2.6329Start save model epoch  122
Optimizer learning rate =  1.5833612e-05
End save model epoch  122
19417/19417 [==============================] - 3792s 195ms/step - loss: 2.6329 - val_loss: 2.6110
Epoch 124/131
12679/19417 [==================>...........] - ETA: 20:45 - loss: 2.6258


Epoch 124/153
19417/19417 [==============================] - ETA: 0s - loss: 2.7291Start save model epoch  123
Optimizer learning rate =  1.5555835e-05
End save model epoch  123
19417/19417 [==============================] - 3722s 187ms/step - loss: 2.7291 - val_loss: 2.6036
Epoch 125/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6779Start save model epoch  124
Optimizer learning rate =  1.527806e-05
End save model epoch  124
19417/19417 [==============================] - 3659s 188ms/step - loss: 2.6779 - val_loss: 2.5974
Epoch 126/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6327Start save model epoch  125
Optimizer learning rate =  1.5000284e-05
End save model epoch  125
19417/19417 [==============================] - 3687s 190ms/step - loss: 2.6327 - val_loss: 2.5941
Epoch 127/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6227Start save model epoch  126
Optimizer learning rate =  1.4722508e-05
End save model epoch  126
19417/19417 [==============================] - 3695s 190ms/step - loss: 2.6227 - val_loss: 2.5875
Epoch 128/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6164Start save model epoch  127
Optimizer learning rate =  1.4444734e-05
End save model epoch  127
19417/19417 [==============================] - 3612s 186ms/step - loss: 2.6164 - val_loss: 2.6912
Epoch 129/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5877Start save model epoch  128
Optimizer learning rate =  1.4166958e-05
End save model epoch  128
19417/19417 [==============================] - 3687s 190ms/step - loss: 2.5877 - val_loss: 2.5761
Epoch 130/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6098Start save model epoch  129
Optimizer learning rate =  1.3889182e-05
End save model epoch  129
19417/19417 [==============================] - 3704s 191ms/step - loss: 2.6098 - val_loss: 2.6102
Epoch 131/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6081Start save model epoch  130
Optimizer learning rate =  1.3611407e-05
End save model epoch  130
19417/19417 [==============================] - 3694s 190ms/step - loss: 2.6081 - val_loss: 2.5378
Epoch 132/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6138Start save model epoch  131
Optimizer learning rate =  1.3333631e-05
End save model epoch  131
19417/19417 [==============================] - 3746s 193ms/step - loss: 2.6138 - val_loss: 2.5952
Epoch 133/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6149Start save model epoch  132
Optimizer learning rate =  1.3055856e-05
End save model epoch  132
19417/19417 [==============================] - 3705s 191ms/step - loss: 2.6149 - val_loss: 2.5825
Epoch 134/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5660Start save model epoch  133
Optimizer learning rate =  1.27780795e-05
End save model epoch  133
19417/19417 [==============================] - 3647s 188ms/step - loss: 2.5660 - val_loss: 2.5727
Epoch 135/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5875Start save model epoch  134
Optimizer learning rate =  1.2500304e-05
End save model epoch  134
19417/19417 [==============================] - 3684s 190ms/step - loss: 2.5875 - val_loss: 2.5692
Epoch 136/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5873Start save model epoch  135
Optimizer learning rate =  1.2222528e-05
End save model epoch  135
19417/19417 [==============================] - 3643s 188ms/step - loss: 2.5873 - val_loss: 2.5685
Epoch 137/153
19417/19417 [==============================] - ETA: 0s - loss: 2.6157Start save model epoch  136
Optimizer learning rate =  1.1944752e-05
End save model epoch  136
19417/19417 [==============================] - 3638s 187ms/step - loss: 2.6157 - val_loss: 2.4873
Epoch 138/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5772Start save model epoch  137
Optimizer learning rate =  1.1666977e-05
End save model epoch  137
19417/19417 [==============================] - 3670s 189ms/step - loss: 2.5772 - val_loss: 2.5088
Epoch 139/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5666Start save model epoch  138
Optimizer learning rate =  1.1389201e-05
End save model epoch  138
19417/19417 [==============================] - 3736s 192ms/step - loss: 2.5666 - val_loss: 2.5121
Epoch 140/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5474Start save model epoch  139
Optimizer learning rate =  1.1111426e-05
End save model epoch  139
19417/19417 [==============================] - 3705s 191ms/step - loss: 2.5474 - val_loss: 2.4772
Epoch 141/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5399Start save model epoch  140
Optimizer learning rate =  1.083365e-05
End save model epoch  140
19417/19417 [==============================] - 3723s 192ms/step - loss: 2.5399 - val_loss: 2.4897
Epoch 142/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5750Start save model epoch  141
Optimizer learning rate =  1.0555874e-05
End save model epoch  141
19417/19417 [==============================] - 3680s 190ms/step - loss: 2.5750 - val_loss: 2.5112
Epoch 143/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5509Start save model epoch  142
Optimizer learning rate =  1.0278099e-05
End save model epoch  142
19417/19417 [==============================] - 3704s 191ms/step - loss: 2.5509 - val_loss: 2.5517
Epoch 144/153
19417/19417 [==============================] - ETA: 0s - loss: 2.4983Start save model epoch  143
Optimizer learning rate =  1.0000323e-05
End save model epoch  143
19417/19417 [==============================] - 3695s 190ms/step - loss: 2.4983 - val_loss: 2.5147
Epoch 145/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5315Start save model epoch  144
Optimizer learning rate =  9.7225475e-06
End save model epoch  144
19417/19417 [==============================] - 3744s 193ms/step - loss: 2.5315 - val_loss: 2.4812
Epoch 146/153
19417/19417 [==============================] - ETA: 0s - loss: 2.5185Start save model epoch  145
Optimizer learning rate =  9.444771e-06
End save model epoch  145
19417/19417 [==============================] - 3698s 190ms/step - loss: 2.5185 - val_loss: 2.4442
Epoch 147/153
 1208/19417 [>.............................] - ETA: 55:56 - loss: 2.4164

Epoch 147/176
19417/19417 [==============================] - ETA: 0s - loss: 2.5787Start save model epoch  146
Optimizer learning rate =  9.166996e-06
End save model epoch  146
19417/19417 [==============================] - 3982s 200ms/step - loss: 2.5787 - val_loss: 2.5157
Epoch 148/176
19417/19417 [==============================] - ETA: 0s - loss: 2.5139Start save model epoch  147
Optimizer learning rate =  8.889221e-06
End save model epoch  147
19417/19417 [==============================] - 3778s 195ms/step - loss: 2.5139 - val_loss: 2.4974
Epoch 149/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4888Start save model epoch  148
Optimizer learning rate =  8.611445e-06
End save model epoch  148
19417/19417 [==============================] - 3731s 192ms/step - loss: 2.4888 - val_loss: 2.4955
Epoch 150/176
19417/19417 [==============================] - ETA: 0s - loss: 2.5033Start save model epoch  149
Optimizer learning rate =  8.33367e-06
End save model epoch  149
19417/19417 [==============================] - 3730s 192ms/step - loss: 2.5033 - val_loss: 2.4443
Epoch 151/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4996Start save model epoch  150
Optimizer learning rate =  8.055894e-06
End save model epoch  150
19417/19417 [==============================] - 3767s 194ms/step - loss: 2.4996 - val_loss: 2.4422
Epoch 152/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4863Start save model epoch  151
Optimizer learning rate =  7.778119e-06
End save model epoch  151
19417/19417 [==============================] - 3743s 193ms/step - loss: 2.4863 - val_loss: 2.4802
Epoch 153/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4768Start save model epoch  152
Optimizer learning rate =  7.5003427e-06
End save model epoch  152
19417/19417 [==============================] - 3731s 192ms/step - loss: 2.4768 - val_loss: 2.5193
Epoch 154/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4801Start save model epoch  153
Optimizer learning rate =  7.222567e-06
End save model epoch  153
19417/19417 [==============================] - 3750s 193ms/step - loss: 2.4801 - val_loss: 2.5147
Epoch 155/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4708Start save model epoch  154
Optimizer learning rate =  6.944792e-06
End save model epoch  154
19417/19417 [==============================] - 3750s 193ms/step - loss: 2.4708 - val_loss: 2.4965
Epoch 156/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4737Start save model epoch  155
Optimizer learning rate =  6.667016e-06
End save model epoch  155
19417/19417 [==============================] - 3696s 190ms/step - loss: 2.4737 - val_loss: 2.5381
Epoch 157/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4855Start save model epoch  156
Optimizer learning rate =  6.38924e-06
End save model epoch  156
19417/19417 [==============================] - 3741s 193ms/step - loss: 2.4855 - val_loss: 2.4687
Epoch 158/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4668Start save model epoch  157
Optimizer learning rate =  6.1114647e-06
End save model epoch  157
19417/19417 [==============================] - 3752s 193ms/step - loss: 2.4668 - val_loss: 2.4587
Epoch 159/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4669Start save model epoch  158
Optimizer learning rate =  5.833689e-06
End save model epoch  158
19417/19417 [==============================] - 3763s 194ms/step - loss: 2.4669 - val_loss: 2.4625
Epoch 160/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4818Start save model epoch  159
Optimizer learning rate =  5.555913e-06
End save model epoch  159
19417/19417 [==============================] - 4071s 210ms/step - loss: 2.4818 - val_loss: 2.4438
Epoch 161/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4622Start save model epoch  160
Optimizer learning rate =  5.278138e-06
End save model epoch  160
19417/19417 [==============================] - 4080s 210ms/step - loss: 2.4622 - val_loss: 2.4732
Epoch 162/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4552Start save model epoch  161
Optimizer learning rate =  5.000362e-06
End save model epoch  161
19417/19417 [==============================] - 3992s 206ms/step - loss: 2.4552 - val_loss: 2.4237
Epoch 163/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4540Start save model epoch  162
Optimizer learning rate =  4.7225863e-06
End save model epoch  162
19417/19417 [==============================] - 4040s 208ms/step - loss: 2.4540 - val_loss: 2.4613
Epoch 164/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4402Start save model epoch  163
Optimizer learning rate =  4.4448107e-06
End save model epoch  163
19417/19417 [==============================] - 3965s 204ms/step - loss: 2.4402 - val_loss: 2.5003
Epoch 165/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4300Start save model epoch  164
Optimizer learning rate =  4.167035e-06
End save model epoch  164
19417/19417 [==============================] - 3982s 205ms/step - loss: 2.4300 - val_loss: 2.4033
Epoch 166/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4334Start save model epoch  165
Optimizer learning rate =  3.88926e-06
End save model epoch  165
19417/19417 [==============================] - 4068s 210ms/step - loss: 2.4334 - val_loss: 2.4486
Epoch 167/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4172Start save model epoch  166
Optimizer learning rate =  3.6114836e-06
End save model epoch  166
19417/19417 [==============================] - 4071s 210ms/step - loss: 2.4172 - val_loss: 2.4345
Epoch 168/176
19417/19417 [==============================] - ETA: 0s - loss: 2.4645Start save model epoch  167
Optimizer learning rate =  3.3337083e-06
End save model epoch  167
19417/19417 [==============================] - 4016s 207ms/step - loss: 2.4645 - val_loss: 2.4371
Epoch 169/176
  568/19417 [..............................] - ETA: 1:02:27 - loss: 2.4320

Epoch 169/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4611Start save model epoch  168
Optimizer learning rate =  3.0559327e-06
End save model epoch  168
19417/19417 [==============================] - 3877s 195ms/step - loss: 2.4611 - val_loss: 2.4056
Epoch 170/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4573Start save model epoch  169
Optimizer learning rate =  2.7781568e-06
End save model epoch  169
19417/19417 [==============================] - 3748s 193ms/step - loss: 2.4573 - val_loss: 2.4147
Epoch 171/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4252Start save model epoch  170
Optimizer learning rate =  2.5003812e-06
End save model epoch  170
19417/19417 [==============================] - 3730s 192ms/step - loss: 2.4252 - val_loss: 2.4072
Epoch 172/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4164Start save model epoch  171
Optimizer learning rate =  2.2226056e-06
End save model epoch  171
19417/19417 [==============================] - 3796s 195ms/step - loss: 2.4164 - val_loss: 2.4334
Epoch 173/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4233Start save model epoch  172
Optimizer learning rate =  1.94483e-06
End save model epoch  172
19417/19417 [==============================] - 3764s 194ms/step - loss: 2.4233 - val_loss: 2.4016
Epoch 174/180
19417/19417 [==============================] - ETA: 0s - loss: 2.3982Start save model epoch  173
Optimizer learning rate =  1.6670544e-06
End save model epoch  173
19417/19417 [==============================] - 3744s 193ms/step - loss: 2.3982 - val_loss: 2.4188
Epoch 175/180
19417/19417 [==============================] - ETA: 0s - loss: 2.3970Start save model epoch  174
Optimizer learning rate =  1.3892786e-06
End save model epoch  174
19417/19417 [==============================] - 3765s 194ms/step - loss: 2.3970 - val_loss: 2.4569
Epoch 176/180
19417/19417 [==============================] - ETA: 0s - loss: 2.3990Start save model epoch  175
Optimizer learning rate =  1.111503e-06
End save model epoch  175
19417/19417 [==============================] - 3758s 194ms/step - loss: 2.3990 - val_loss: 2.4486
Epoch 177/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4152Start save model epoch  176
Optimizer learning rate =  8.337273e-07
End save model epoch  176
19417/19417 [==============================] - 3773s 194ms/step - loss: 2.4152 - val_loss: 2.4221
Epoch 178/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4155Start save model epoch  177
Optimizer learning rate =  5.559517e-07
End save model epoch  177
19417/19417 [==============================] - 3757s 193ms/step - loss: 2.4155 - val_loss: 2.3982
Epoch 179/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4143Start save model epoch  178
Optimizer learning rate =  2.7817595e-07
End save model epoch  178
19417/19417 [==============================] - 3733s 192ms/step - loss: 2.4143 - val_loss: 2.3549
Epoch 180/180
19417/19417 [==============================] - ETA: 0s - loss: 2.4073Start save model epoch  179
Optimizer learning rate =  4.0038783e-10
End save model epoch  179
19417/19417 [==============================] - 3739s 193ms/step - loss: 2.4073 - val_loss: 2.4055




"""

'\nEpoch 1/30\n19417/19417 [==============================] - ETA: 0s - loss: 5.7616Start save model epoch  0\nOptimizer learning rate =  4.972224e-05\nEnd save model epoch  0\n19417/19417 [==============================] - 3831s 192ms/step - loss: 5.7616 - val_loss: 5.1976\nEpoch 2/30\n19417/19417 [==============================] - ETA: 0s - loss: 4.9109Start save model epoch  1\nOptimizer learning rate =  4.944446e-05\nEnd save model epoch  1\n19417/19417 [==============================] - 3721s 192ms/step - loss: 4.9109 - val_loss: 4.6028\nEpoch 3/30\n19417/19417 [==============================] - ETA: 0s - loss: 4.5231Start save model epoch  2\nOptimizer learning rate =  4.9166687e-05\nEnd save model epoch  2\n19417/19417 [==============================] - 3679s 189ms/step - loss: 4.5231 - val_loss: 4.3518\nEpoch 4/30\n19417/19417 [==============================] - ETA: 0s - loss: 4.3601Start save model epoch  3\nOptimizer learning rate =  4.8888913e-05\nEnd save model epoch  3\n19

In [ ]:
new_model.evaluate(tf_valid_dataset)

2461/2461 [==============================] - 176s 71ms/step - loss: 2.3936


2.393561601638794

In [ ]:
new_model.evaluate(tf_test_dataset)

2891/2891 [==============================] - 215s 70ms/step - loss: 2.3976


2.397552967071533